# Characterizing CpG Methylation

To describe general metylation trends, irrespective of pCO<sub>2</sub> treatment in *C. virginica* gonad sequence data, I need to characterize individual CpG loci. Gavery and Roberts (2013) and Olson and Roberts (2013) define a CpG locus as methylated if at least half of the reads remained unconverted after bisulfite treatment. I will use information in a master `.cov` file to identify methylated CpG loci.

Another thing I will do is identify methylation islands by replicating [Jeong et al. 2018](https://academic.oup.com/gbe/article/10/10/2766/5098531). I will use [their script](https://github.com/soojinyilab/Methylation-Islands) but modify parameters to reflect differences in insect and *C. virginica* methylation.

1. Download coverage file
2. Limit to 5x coverage
3. Characterize methylation levels for loci
4. Characterize loci locations
5. Identify methylation islands

## 0. Prepare for analyses

## 0a. Set working directory

In [1]:
pwd

'/Users/yaamini/Documents/yaamini-virginica/notebooks'

In [2]:
cd ../analyses/

/Users/yaamini/Documents/yaamini-virginica/analyses


In [3]:
#!mkdir 2019-03-18-Characterizing-CpG-Methylation

In [3]:
cd 2019-03-18-Characterizing-CpG-Methylation/

/Users/yaamini/Documents/yaamini-virginica/analyses/2019-03-18-Characterizing-CpG-Methylation


## 1. Obtain coverage files

In [4]:
#Download file from gannet. This file is a concatenation of coverage and methylation information for all samples
!wget http://gannet.fish.washington.edu/Atumefaciens/20190312_cvir_gonad_bismark/total_reads_bismark/cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz

--2019-04-09 14:41:39--  http://gannet.fish.washington.edu/Atumefaciens/20190312_cvir_gonad_bismark/total_reads_bismark/cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz
Resolving gannet.fish.washington.edu... 128.95.149.52
Connecting to gannet.fish.washington.edu|128.95.149.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94181669 (90M) [application/x-gzip]
Saving to: 'cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz'

cvir_bsseq_all_pe_R 100%[===================>]  89.82M  75.1MB/s    in 1.2s    

2019-04-09 14:41:40 (75.1 MB/s) - 'cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz' saved [94181669/94181669]



In [8]:
#Unzip the coverage file
!gunzip *cov.gz

In [4]:
#Confirm file was unzipped
!ls *cov

cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov


In [4]:
#See what the file looks like. 
#Columns: <chromosome> <start position> <end position> <methylation percentage> <count methylated> <count unmethylated>
!head cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov

NC_007175.2	49	49	1.25	2	158
NC_007175.2	50	50	0	0	15
NC_007175.2	51	51	1.18343195266272	2	167
NC_007175.2	52	52	0	0	18
NC_007175.2	88	88	1.02459016393443	5	483
NC_007175.2	89	89	1.38888888888889	5	355
NC_007175.2	100	100	0	0	1
NC_007175.2	129	129	0	0	1
NC_007175.2	147	147	1.99115044247788	18	886
NC_007175.2	148	148	2.29885057471264	6	255


In [6]:
#See how many loci have data
!awk '{if ($5+$6 >= 1) { print $1, $2-1, $3, $4, $5+$6}}' cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov \
| wc -l

 14026131


14,026,131 CpGs have data, which is close to the 14,458,703 CG motifs in the *C. virginica* genome.

## 2. Limit to 5x coverage

In [11]:
#If total coverage (count methylated + unmethylated) is greater than 5
#then print the chromosome, start pos -1, stop pos, percent methylation, and total coverage
#Save output as new file
!awk '{if ($5+$6 >= 5) { print $1, $2-1, $3, $4, $5+$6}}' cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov \
> 2019-04-09-All-5x-CpGs.bedgraph

In [12]:
#Check columns for one of the file: <chromosome> <start position> <stop position> <percent methylation> <coverage>
!head 2019-04-09-All-5x-CpGs.bedgraph

NC_007175.2 48 49 1.25 160
NC_007175.2 49 50 0 15
NC_007175.2 50 51 1.18343195266272 169
NC_007175.2 51 52 0 18
NC_007175.2 87 88 1.02459016393443 488
NC_007175.2 88 89 1.38888888888889 360
NC_007175.2 146 147 1.99115044247788 904
NC_007175.2 147 148 2.29885057471264 261
NC_007175.2 173 174 0 5
NC_007175.2 192 193 1.25786163522013 795


In [58]:
#Count loci with 5x coverage
!wc -l 2019-04-09-All-5x-CpGs.bedgraph

 4304257 2019-04-09-All-5x-CpGs.bedgraph


I have data for 4,304,257 CpG loci with 5x coverge.

In [59]:
#Replace delimiters to save .bedgraph as .csv
!awk '{print $1","$2","$3","$4 }' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpGs.csv

In [60]:
#Confirm .csv creation
!head 2019-04-09-All-5x-CpGs.csv

NC_007175.2,48,49,1.25
NC_007175.2,49,50,0
NC_007175.2,50,51,1.18343195266272
NC_007175.2,51,52,0
NC_007175.2,87,88,1.02459016393443
NC_007175.2,88,89,1.38888888888889
NC_007175.2,146,147,1.99115044247788
NC_007175.2,147,148,2.29885057471264
NC_007175.2,173,174,0
NC_007175.2,192,193,1.25786163522013


## 3. Characterize methylation levels for loci

Olson and Roberts (2014) define the following categories for CpG methylation:

- Methylated (50% methylation and above)
- Sparsely methylated (0-50% methylated)
- Unmethylated (0% methylation)

I will slightly modify this since I have multiple samples:

- Methylated (50% methylation and above)
- Sparsely methylated (10-50% methylated)
- Unmethylated (10% methylation and below)

### 3a. Methylated loci

In [19]:
#If percent methylation is greater or equal to 50, then save the loci information
!awk '{if ($4 >= 50) { print $1, $2, $3, $4 }}' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph

In [48]:
#Confirm methylated loci were saved
!head 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph

NC_035780.1 9253 9254 60
NC_035780.1 9637 9638 60
NC_035780.1 9657 9658 50
NC_035780.1 10089 10090 71.4285714285714
NC_035780.1 10331 10332 80
NC_035780.1 11692 11693 80
NC_035780.1 11706 11707 80
NC_035780.1 11711 11712 80
NC_035780.1 12686 12687 69.2307692307692
NC_035780.1 12758 12759 80


In [21]:
#Count methylated loci
!wc -l 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph

 3181904 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph


In [53]:
#Replace delimiters to save .bedgraph as .csv
!awk '{print $1","$2","$3","$4 }' 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Methylated.csv

In [54]:
#Check .csv was saved
!head 2019-04-09-All-5x-CpG-Loci-Methylated.csv

NC_035780.1,9253,9254,60
NC_035780.1,9637,9638,60
NC_035780.1,9657,9658,50
NC_035780.1,10089,10090,71.4285714285714
NC_035780.1,10331,10332,80
NC_035780.1,11692,11693,80
NC_035780.1,11706,11707,80
NC_035780.1,11711,11712,80
NC_035780.1,12686,12687,69.2307692307692
NC_035780.1,12758,12759,80


### 3b. Sparsely methylated loci

In [33]:
%%bash
awk '{if ($4 < 50) { print $1, $2, $3, $4}}' 2019-04-09-All-5x-CpGs.bedgraph \
| awk '{if ($4 > 10) { print $1, $2, $3, $4 }}' \
> 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph

In [34]:
#Confirm sparsely methylated loci were saved
!head 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph

NC_007175.2 1506 1507 16.6666666666667
NC_007175.2 1820 1821 20
NC_007175.2 2128 2129 11.7647058823529
NC_007175.2 4841 4842 15
NC_007175.2 13069 13070 20
NC_035780.1 421 422 14.2857142857143
NC_035780.1 1101 1102 12.5
NC_035780.1 1540 1541 16.6666666666667
NC_035780.1 3468 3469 16.6666666666667
NC_035780.1 9254 9255 28.5714285714286


In [35]:
#Count sparsely methylated loci
!wc -l 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph

  481788 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph


### 3c. Unmethylated loci

In [36]:
!awk '{if ($4 <= 10) { print $1, $2, $3, $4 }}' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph

In [37]:
#Confirm unmethylated loci were saved
!head 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph

NC_007175.2 48 49 1.25
NC_007175.2 49 50 0
NC_007175.2 50 51 1.18343195266272
NC_007175.2 51 52 0
NC_007175.2 87 88 1.02459016393443
NC_007175.2 88 89 1.38888888888889
NC_007175.2 146 147 1.99115044247788
NC_007175.2 147 148 2.29885057471264
NC_007175.2 173 174 0
NC_007175.2 192 193 1.25786163522013


In [38]:
#Count unmethylated loci
!wc -l 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph

  640565 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph


## 4. Characterize loci locations

My final step is to characterize the location of various loci categories in the genome. I will use `intersectBed` to find overlaps between all 5x CpGs, methylated loci, sparsely methylated loci, and unmethylated loci with exons, introns, mRNA coding regions, transposable elements, and putative promoter regions.

### 4a. Create `.bed` files

#### All 5x CpGs

In [69]:
%%bash
awk '{print $1"\t"$2"\t"$3}' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpGs.bed

In [70]:
#Confirm file creation
!head 2019-04-09-All-5x-CpGs.bed

NC_007175.2	48	49
NC_007175.2	49	50
NC_007175.2	50	51
NC_007175.2	51	52
NC_007175.2	87	88
NC_007175.2	88	89
NC_007175.2	146	147
NC_007175.2	147	148
NC_007175.2	173	174
NC_007175.2	192	193


#### Methylated loci

In [8]:
%%bash
awk '{print $1"\t"$2"\t"$3}' 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Methylated.bed

In [4]:
#Confirm file creation
!head 2019-04-09-All-5x-CpG-Loci-Methylated.bed

NC_035780.1	9253	9254
NC_035780.1	9637	9638
NC_035780.1	9657	9658
NC_035780.1	10089	10090
NC_035780.1	10331	10332
NC_035780.1	11692	11693
NC_035780.1	11706	11707
NC_035780.1	11711	11712
NC_035780.1	12686	12687
NC_035780.1	12758	12759


#### Sparsely methylated loci

In [5]:
%%bash
awk '{print $1"\t"$2"\t"$3}' 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bed

In [6]:
#Confirm file creation
!head 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bed

NC_007175.2	1506	1507
NC_007175.2	1820	1821
NC_007175.2	2128	2129
NC_007175.2	4841	4842
NC_007175.2	13069	13070
NC_035780.1	421	422
NC_035780.1	1101	1102
NC_035780.1	1540	1541
NC_035780.1	3468	3469
NC_035780.1	9254	9255


#### Unmethylated loci

In [7]:
%%bash
awk '{print $1"\t"$2"\t"$3}' 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Unmethylated.bed

In [8]:
#Confirm file creation
!head 2019-04-09-All-5x-CpG-Loci-Unmethylated.bed

NC_007175.2	48	49
NC_007175.2	49	50
NC_007175.2	50	51
NC_007175.2	51	52
NC_007175.2	87	88
NC_007175.2	88	89
NC_007175.2	146	147
NC_007175.2	147	148
NC_007175.2	173	174
NC_007175.2	192	193


### 4b. Set variable paths

In [23]:
bedtoolsDirectory = "/Users/yaamini/bedtools2/bin/"

In [6]:
all5xCpGs = "2019-04-09-All-5x-CpGs.bed"

In [7]:
methylatedLoci = "2019-04-09-All-5x-CpG-Loci-Methylated.bed"

In [8]:
sparselyMethylatedLoci = "2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bed"

In [9]:
unmethylatedLoci = "2019-04-09-All-5x-CpG-Loci-Unmethylated.bed"

In [10]:
exonList = "../2018-11-01-DML-and-DMR-Analysis/C_virginica-3.0_Gnomon_exon_sorted_yrv.bed"

In [11]:
intronList = "../2018-11-01-DML-and-DMR-Analysis/C_virginica-3.0_Gnomon_intron_yrv.bed"

In [12]:
geneList = "../2018-11-01-DML-and-DMR-Analysis/C_virginica-3.0_Gnomon_gene_sorted_yrv.bed"

In [13]:
transposableElementsAll = "../2018-11-01-DML-and-DMR-Analysis/C_virginica-3.0_TE-all.gff"

In [14]:
transposableElementsCg = "../2018-11-01-DML-and-DMR-Analysis/C_virginica-3.0_TE-Cg.gff"

In [154]:
mRNAList = "../2019-05-13-Generating-Genome-Feature-Tracks/C_virginica-3.0_Gnomon_mRNA_yrv.gff3"

In [15]:
putativePromoters = "../2018-11-01-DML-and-DMR-Analysis/2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Promoter-Track.bed"

In [16]:
exonUTR = "../2019-05-13-Generating-Genome-Feature-Tracks/C_virginica-3.0_Gnomon_exonUTR_yrv.gff3"

In [17]:
CDS = "../2019-05-13-Generating-Genome-Feature-Tracks/C_virginica-3.0_Gnomon_CSD_sorted_yrv.bed"

In [20]:
nonCDS = "../2019-05-13-Generating-Genome-Feature-Tracks/C_virginica-3.0_Gnomon_noncoding_yrv.gff3"

In [21]:
lncRNA = "../2019-05-13-Generating-Genome-Feature-Tracks/C_virginica-3.0_Gnomon_lncRNA_yrv.gff3"

In [18]:
intergenic = "../2019-05-13-Generating-Genome-Feature-Tracks/C_virginica-3.0_Gnomon_intergenic_yrv.gff3"

### 4c. Exons

#### All 5x CpGs

In [16]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {exonList} \
| wc -l
!echo "all 5x CpG loci overlaps with exons"

 1366779
all 5x CpG loci overlaps with exons


In [17]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {exonList} \
> 2019-05-29-All5xCpGs-Exon.txt

In [18]:
!head 2019-05-29-All5xCpGs-Exon.txt

NC_035780.1	28992	28993	NC_035780.1	28961	29073
NC_035780.1	29001	29002	NC_035780.1	28961	29073
NC_035780.1	30723	30724	NC_035780.1	30524	31557
NC_035780.1	30765	30766	NC_035780.1	30524	31557
NC_035780.1	30811	30812	NC_035780.1	30524	31557
NC_035780.1	30906	30907	NC_035780.1	30524	31557
NC_035780.1	30932	30933	NC_035780.1	30524	31557
NC_035780.1	30935	30936	NC_035780.1	30524	31557
NC_035780.1	31017	31018	NC_035780.1	30524	31557
NC_035780.1	31018	31019	NC_035780.1	30524	31557


#### Methylated loci

In [19]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {exonList} \
| wc -l
!echo "methylated loci overlaps with exons"

 1013691
methylated loci overlaps with exons


In [20]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {exonList} \
> 2019-05-29-MethLoci-Exon.txt

In [21]:
!head 2019-05-29-MethLoci-Exon.txt

NC_035780.1	100558	100559	NC_035780.1	100554	100661
NC_035780.1	100559	100560	NC_035780.1	100554	100661
NC_035780.1	100575	100576	NC_035780.1	100554	100661
NC_035780.1	100576	100577	NC_035780.1	100554	100661
NC_035780.1	100581	100582	NC_035780.1	100554	100661
NC_035780.1	100582	100583	NC_035780.1	100554	100661
NC_035780.1	100634	100635	NC_035780.1	100554	100661
NC_035780.1	100635	100636	NC_035780.1	100554	100661
NC_035780.1	100643	100644	NC_035780.1	100554	100661
NC_035780.1	100644	100645	NC_035780.1	100554	100661


#### Sparsely methylated loci

In [22]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {exonList} \
| wc -l
!echo "sparsely methylated loci overlaps with exons"

  105871
sparsely methylated loci overlaps with exons


In [23]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {exonList} \
> 2019-05-29-SparseMethLoci-Exon.txt

In [24]:
!head 2019-05-29-SparseMethLoci-Exon.txt

NC_035780.1	31078	31079	NC_035780.1	30524	31557
NC_035780.1	85755	85756	NC_035780.1	85606	85777
NC_035780.1	94754	94755	NC_035780.1	94571	95254
NC_035780.1	106236	106237	NC_035780.1	106004	106460
NC_035780.1	204528	204529	NC_035780.1	204243	204795
NC_035780.1	207401	207402	NC_035780.1	207388	207743
NC_035780.1	207423	207424	NC_035780.1	207388	207743
NC_035780.1	207472	207473	NC_035780.1	207388	207743
NC_035780.1	223409	223410	NC_035780.1	223311	223637
NC_035780.1	223416	223417	NC_035780.1	223311	223637


#### Unmethylated loci

In [25]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {exonList} \
| wc -l
!echo "unmethylated loci overlaps with exons"

  247217
unmethylated loci overlaps with exons


In [26]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {exonList} \
> 2019-05-29-UnMethLoci-Exon.txt

In [27]:
!head 2019-05-29-UnMethLoci-Exon.txt

NC_035780.1	28992	28993	NC_035780.1	28961	29073
NC_035780.1	29001	29002	NC_035780.1	28961	29073
NC_035780.1	30723	30724	NC_035780.1	30524	31557
NC_035780.1	30765	30766	NC_035780.1	30524	31557
NC_035780.1	30811	30812	NC_035780.1	30524	31557
NC_035780.1	30906	30907	NC_035780.1	30524	31557
NC_035780.1	30932	30933	NC_035780.1	30524	31557
NC_035780.1	30935	30936	NC_035780.1	30524	31557
NC_035780.1	31017	31018	NC_035780.1	30524	31557
NC_035780.1	31018	31019	NC_035780.1	30524	31557


### 4d. Introns

#### All 5x CpG

In [28]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {intronList} \
| wc -l
!echo "all 5x CpG loci overlaps with introns"

 1884429
all 5x CpG loci overlaps with introns


In [29]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {intronList} \
> 2019-05-29-All5xCpGs-Intron.txt

In [30]:
!head 2019-05-29-All5xCpGs-Intron.txt

NC_035780.1	29412	29413	NC_035780.1	29073	30523
NC_035780.1	31940	31941	NC_035780.1	31887	31976
NC_035780.1	44372	44373	NC_035780.1	44358	45912
NC_035780.1	45142	45143	NC_035780.1	44358	45912
NC_035780.1	45542	45543	NC_035780.1	44358	45912
NC_035780.1	46515	46516	NC_035780.1	46506	64122
NC_035780.1	47583	47584	NC_035780.1	46506	64122
NC_035780.1	47590	47591	NC_035780.1	46506	64122
NC_035780.1	47651	47652	NC_035780.1	46506	64122
NC_035780.1	47679	47680	NC_035780.1	46506	64122


#### Methylated loci

In [31]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {intronList} \
| wc -l
!echo "methylated loci overlaps with introns"

 1504791
methylated loci overlaps with introns


In [32]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {intronList} \
> 2019-05-29-MethLoci-Intron.txt

In [33]:
!head 2019-05-29-MethLoci-Intron.txt

NC_035780.1	29412	29413	NC_035780.1	29073	30523
NC_035780.1	87531	87532	NC_035780.1	85777	88422
NC_035780.1	87541	87542	NC_035780.1	85777	88422
NC_035780.1	87590	87591	NC_035780.1	85777	88422
NC_035780.1	87595	87596	NC_035780.1	85777	88422
NC_035780.1	100664	100665	NC_035780.1	100661	104928
NC_035780.1	100665	100666	NC_035780.1	100661	104928
NC_035780.1	100917	100918	NC_035780.1	100661	104928
NC_035780.1	100975	100976	NC_035780.1	100661	104928
NC_035780.1	101305	101306	NC_035780.1	100661	104928


#### Sparsely methylated loci

In [34]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {intronList} \
| wc -l
!echo "sparsely methylated loci overlaps with introns"

  211143
sparsely methylated loci overlaps with introns


In [35]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {intronList} \
> 2019-05-29-SparseMethLoci-Intron.txt

In [36]:
!head 2019-05-29-SparseMethLoci-Intron.txt

NC_035780.1	45142	45143	NC_035780.1	44358	45912
NC_035780.1	45542	45543	NC_035780.1	44358	45912
NC_035780.1	48914	48915	NC_035780.1	46506	64122
NC_035780.1	48928	48929	NC_035780.1	46506	64122
NC_035780.1	48940	48941	NC_035780.1	46506	64122
NC_035780.1	87599	87600	NC_035780.1	85777	88422
NC_035780.1	87607	87608	NC_035780.1	85777	88422
NC_035780.1	103272	103273	NC_035780.1	100661	104928
NC_035780.1	104332	104333	NC_035780.1	100661	104928
NC_035780.1	105767	105768	NC_035780.1	105614	106003


#### Unmethylated loci

In [37]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {intronList} \
| wc -l
!echo "unmethylated loci overlaps with introns"

  168495
unmethylated loci overlaps with introns


In [38]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {intronList} \
> 2019-05-29-UnMethLoci-Intron.txt

In [39]:
!head 2019-05-29-UnMethLoci-Intron.txt

NC_035780.1	31940	31941	NC_035780.1	31887	31976
NC_035780.1	44372	44373	NC_035780.1	44358	45912
NC_035780.1	46515	46516	NC_035780.1	46506	64122
NC_035780.1	47583	47584	NC_035780.1	46506	64122
NC_035780.1	47590	47591	NC_035780.1	46506	64122
NC_035780.1	47651	47652	NC_035780.1	46506	64122
NC_035780.1	47679	47680	NC_035780.1	46506	64122
NC_035780.1	48094	48095	NC_035780.1	46506	64122
NC_035780.1	48108	48109	NC_035780.1	46506	64122
NC_035780.1	48114	48115	NC_035780.1	46506	64122


### 4e. Exon UTR

#### All 5x CpGs

In [24]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {exonUTR} \
| wc -l
!echo "all 5x CpG loci overlaps with exon UTR"

  192907
all 5x CpG loci overlaps with exon UTR


In [28]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {exonUTR} \
> 2019-05-29-All5xCpGs-ExonUTR.txt

In [29]:
!head 2019-05-29-All5xCpGs-ExonUTR.txt

NC_035780.1	28992	28993	NC_035780.1	Gnomon	exon	28961	29073	.	+	.	ID=id4;Parent=rna1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;gbkey=mRNA;gene=LOC111126949;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	29001	29002	NC_035780.1	Gnomon	exon	28961	29073	.	+	.	ID=id4;Parent=rna1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;gbkey=mRNA;gene=LOC111126949;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	46002	46003	NC_035780.1	Gnomon	exon	45998	46506	.	-	.	ID=id12;Parent=rna3;Dbxref=GeneID:111110729,Genbank:XM_022447333.1;gbkey=mRNA;gene=LOC111110729;product=FMRFamide receptor-like%2C transcript variant X2;transcript_id=XM_022447333.1
NC_035780.1	46008	46009	NC_035780.1	Gnomon	exon	45998	46506	.	-	.	ID=id12;Parent=rna3;Dbxref=GeneID:111110729,Genbank:XM_022447333.1;gbkey=mRNA;gene=LOC111110729;product=FMRFamide receptor-like%2C transcript variant X2;transcript_id=XM_022447333.1
NC_035780.1	64220	64221	NC_035780.1	Gnomon	exon	64220	64334	.	-	.	ID=i

#### Methylated loci

In [33]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {exonUTR} \
| wc -l
!echo "methylated loci overlaps with exon UTR"

  128585
methylated loci overlaps with exon UTR


In [34]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {exonUTR} \
> 2019-05-29-MethLoci-ExonUTR.txt

In [35]:
!head 2019-05-29-MethLoci-ExonUTR.txt

NC_035780.1	106306	106307	NC_035780.1	Gnomon	exon	106121	106460	.	+	.	ID=id21;Parent=rna5;Dbxref=GeneID:111120752,Genbank:XM_022461698.1;gbkey=mRNA;gene=LOC111120752;product=ribulose-phosphate 3-epimerase-like;transcript_id=XM_022461698.1
NC_035780.1	164863	164864	NC_035780.1	Gnomon	exon	164820	164941	.	+	.	ID=id27;Parent=rna9;Dbxref=GeneID:111105685,Genbank:XM_022440042.1;gbkey=mRNA;gene=LOC111105685;product=protein ANTAGONIST OF LIKE HETEROCHROMATIN PROTEIN 1-like;transcript_id=XM_022440042.1
NC_035780.1	164869	164870	NC_035780.1	Gnomon	exon	164820	164941	.	+	.	ID=id27;Parent=rna9;Dbxref=GeneID:111105685,Genbank:XM_022440042.1;gbkey=mRNA;gene=LOC111105685;product=protein ANTAGONIST OF LIKE HETEROCHROMATIN PROTEIN 1-like;transcript_id=XM_022440042.1
NC_035780.1	165727	165728	NC_035780.1	Gnomon	exon	165620	165745	.	+	.	ID=id28;Parent=rna9;Dbxref=GeneID:111105685,Genbank:XM_022440042.1;gbkey=mRNA;gene=LOC111105685;product=protein ANTAGONIST OF LIKE HETEROCHROMATIN PROTEIN 1-like;transcr

#### Sparsely methylated loci

In [36]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {exonUTR} \
| wc -l
!echo "sparsely methylated loci overlaps with exon UTR"

   19280
sparsely methylated loci overlaps with exon UTR


In [37]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {exonUTR} \
> 2019-05-29-SparseMethLoci-ExonUTR.txt

In [38]:
!head 2019-05-29-SparseMethLoci-ExonUTR.txt

NC_035780.1	106236	106237	NC_035780.1	Gnomon	exon	106121	106460	.	+	.	ID=id21;Parent=rna5;Dbxref=GeneID:111120752,Genbank:XM_022461698.1;gbkey=mRNA;gene=LOC111120752;product=ribulose-phosphate 3-epimerase-like;transcript_id=XM_022461698.1
NC_035780.1	207401	207402	NC_035780.1	Gnomon	exon	207388	207743	.	-	.	ID=id35;Parent=rna12;Dbxref=GeneID:111125466,Genbank:XM_022469388.1;gbkey=mRNA;gene=LOC111125466;product=homeobox protein 2-like;transcript_id=XM_022469388.1
NC_035780.1	207423	207424	NC_035780.1	Gnomon	exon	207388	207743	.	-	.	ID=id35;Parent=rna12;Dbxref=GeneID:111125466,Genbank:XM_022469388.1;gbkey=mRNA;gene=LOC111125466;product=homeobox protein 2-like;transcript_id=XM_022469388.1
NC_035780.1	207472	207473	NC_035780.1	Gnomon	exon	207388	207743	.	-	.	ID=id35;Parent=rna12;Dbxref=GeneID:111125466,Genbank:XM_022469388.1;gbkey=mRNA;gene=LOC111125466;product=homeobox protein 2-like;transcript_id=XM_022469388.1
NC_035780.1	245545	245546	NC_035780.1	Gnomon	exon	245532	245768	.	-	.	ID=id70

#### Unmethylated loci

In [39]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {exonUTR} \
| wc -l
!echo "unmethylated loci overlaps with exon UTR"

   45042
unmethylated loci overlaps with exon UTR


In [40]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {exonUTR} \
> 2019-05-29-UnMethLoci-ExonUTR.txt

In [41]:
!head 2019-05-29-UnMethLoci-ExonUTR.txt

NC_035780.1	28992	28993	NC_035780.1	Gnomon	exon	28961	29073	.	+	.	ID=id4;Parent=rna1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;gbkey=mRNA;gene=LOC111126949;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	29001	29002	NC_035780.1	Gnomon	exon	28961	29073	.	+	.	ID=id4;Parent=rna1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;gbkey=mRNA;gene=LOC111126949;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	46002	46003	NC_035780.1	Gnomon	exon	45998	46506	.	-	.	ID=id12;Parent=rna3;Dbxref=GeneID:111110729,Genbank:XM_022447333.1;gbkey=mRNA;gene=LOC111110729;product=FMRFamide receptor-like%2C transcript variant X2;transcript_id=XM_022447333.1
NC_035780.1	46008	46009	NC_035780.1	Gnomon	exon	45998	46506	.	-	.	ID=id12;Parent=rna3;Dbxref=GeneID:111110729,Genbank:XM_022447333.1;gbkey=mRNA;gene=LOC111110729;product=FMRFamide receptor-like%2C transcript variant X2;transcript_id=XM_022447333.1
NC_035780.1	64220	64221	NC_035780.1	Gnomon	exon	64220	64334	.	-	.	ID=i

### 4f. mRNA

#### All 5x CpG

In [155]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {mRNAList} \
| wc -l
!echo "all 5x CpG loci overlaps with mRNA"

 3140744
all 5x CpG loci overlaps with mRNA


In [156]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {mRNAList} \
> 2019-05-29-All5xCpGs-mRNA.txt

In [157]:
!head 2019-05-29-All5xCpGs-mRNA.txt

NC_035780.1	28992	28993	NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	29001	29002	NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	29412	29413	NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:1

#### Methylated loci

In [158]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {mRNAList} \
| wc -l
!echo "methylated loci overlaps with mRNA"

 2437901
methylated loci overlaps with mRNA


In [159]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {mRNAList} \
> 2019-05-29-MethLoci-mRNA.txt

In [160]:
!head 2019-05-29-MethLoci-mRNA.txt

NC_035780.1	29412	29413	NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	87531	87532	NC_035780.1	Gnomon	mRNA	85606	95254	.	-	.	ID=rna4;Parent=gene3;Dbxref=GeneID:111112434,Genbank:XM_022449924.1;Name=XM_022449924.1;gbkey=mRNA;gene=LOC111112434;model_evidence=Supporting evidence includes similarity to: 7 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 13 samples with support for all annotated introns;product=homeobox protein Hox-B7-like;transcript_id=XM_022449924.1
NC_035780.1	87541	87542	NC_035780.1	Gnomon	mRNA	85606	95254	.	-	.	ID=rna4;Parent=gene3;Dbxre

#### Sparsely methylated loci

In [161]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {mRNAList} \
| wc -l
!echo "sparsely methylated loci overlaps with mRNA"

  303890
sparsely methylated loci overlaps with mRNA


In [162]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {mRNAList} \
> 2019-05-29-SparseMethLoci-mRNA.txt

In [163]:
!head 2019-05-29-SparseMethLoci-mRNA.txt

NC_035780.1	31078	31079	NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	45142	45143	NC_035780.1	Gnomon	mRNA	43111	46506	.	-	.	ID=rna3;Parent=gene2;Dbxref=GeneID:111110729,Genbank:XM_022447333.1;Name=XM_022447333.1;gbkey=mRNA;gene=LOC111110729;model_evidence=Supporting evidence includes similarity to: 1 Protein%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 14 samples with support for all annotated introns;product=FMRFamide receptor-like%2C transcript variant X2;transcript_id=XM_022447333.1
NC_035780.1	45142	45143	NC_035780.1	Gnomon	mRNA	43111	66897	.	-	.	ID=rna2

#### Unmethylated loci

In [164]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {mRNAList} \
| wc -l
!echo "unmethylated loci overlaps with mRNA"

  398953
unmethylated loci overlaps with mRNA


In [165]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {mRNAList} \
> 2019-05-29-UnMethLoci-mRNA.txt

In [166]:
!head 2019-05-29-UnMethLoci-mRNA.txt

NC_035780.1	28992	28993	NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	29001	29002	NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	30723	30724	NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:1

### 4g. Coding sequences

#### All 5x CpGs

In [42]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {CDS} \
| wc -l
!echo "all 5x CpG loci overlaps with CDS"

 1174256
all 5x CpG loci overlaps with CDS


In [43]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {CDS} \
> 2019-05-29-All5xCpGs-CDS.txt

In [44]:
!head 2019-05-29-All5xCpGs-CDS.txt

NC_035780.1	30723	30724	NC_035780.1	30535	31557
NC_035780.1	30765	30766	NC_035780.1	30535	31557
NC_035780.1	30811	30812	NC_035780.1	30535	31557
NC_035780.1	30906	30907	NC_035780.1	30535	31557
NC_035780.1	30932	30933	NC_035780.1	30535	31557
NC_035780.1	30935	30936	NC_035780.1	30535	31557
NC_035780.1	31017	31018	NC_035780.1	30535	31557
NC_035780.1	31018	31019	NC_035780.1	30535	31557
NC_035780.1	31024	31025	NC_035780.1	30535	31557
NC_035780.1	31025	31026	NC_035780.1	30535	31557


#### Methylated loci

In [45]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {CDS} \
| wc -l
!echo "methylated loci overlaps with CDS"

  885327
methylated loci overlaps with CDS


In [46]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {CDS} \
> 2019-05-29-MethLoci-CDS.txt

In [47]:
!head 2019-05-29-MethLoci-CDS.txt

NC_035780.1	100558	100559	NC_035780.1	100554	100661
NC_035780.1	100559	100560	NC_035780.1	100554	100661
NC_035780.1	100575	100576	NC_035780.1	100554	100661
NC_035780.1	100576	100577	NC_035780.1	100554	100661
NC_035780.1	100581	100582	NC_035780.1	100554	100661
NC_035780.1	100582	100583	NC_035780.1	100554	100661
NC_035780.1	100634	100635	NC_035780.1	100554	100661
NC_035780.1	100635	100636	NC_035780.1	100554	100661
NC_035780.1	100643	100644	NC_035780.1	100554	100661
NC_035780.1	100644	100645	NC_035780.1	100554	100661


#### Sparsely methylated loci

In [48]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {CDS} \
| wc -l
!echo "sparsely methylated loci overlaps with CDS"

   86624
sparsely methylated loci overlaps with CDS


In [49]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {CDS} \
> 2019-05-29-SparseMethLoci-CDS.txt

In [50]:
!head 2019-05-29-SparseMethLoci-CDS.txt

NC_035780.1	31078	31079	NC_035780.1	30535	31557
NC_035780.1	85755	85756	NC_035780.1	85616	85777
NC_035780.1	94754	94755	NC_035780.1	94571	95042
NC_035780.1	204528	204529	NC_035780.1	204289	204720
NC_035780.1	223409	223410	NC_035780.1	223311	223637
NC_035780.1	223416	223417	NC_035780.1	223311	223637
NC_035780.1	223445	223446	NC_035780.1	223311	223637
NC_035780.1	245773	245774	NC_035780.1	245769	245878
NC_035780.1	245774	245775	NC_035780.1	245769	245878
NC_035780.1	247058	247059	NC_035780.1	247019	247125


#### Unmethylated loci

In [51]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {CDS} \
| wc -l
!echo "unmethylated loci overlaps with CDS"

  202305
unmethylated loci overlaps with CDS


In [52]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {CDS} \
> 2019-05-29-UnMethLoci-CDS.txt

In [53]:
!head 2019-05-29-UnMethLoci-CDS.txt

NC_035780.1	30723	30724	NC_035780.1	30535	31557
NC_035780.1	30765	30766	NC_035780.1	30535	31557
NC_035780.1	30811	30812	NC_035780.1	30535	31557
NC_035780.1	30906	30907	NC_035780.1	30535	31557
NC_035780.1	30932	30933	NC_035780.1	30535	31557
NC_035780.1	30935	30936	NC_035780.1	30535	31557
NC_035780.1	31017	31018	NC_035780.1	30535	31557
NC_035780.1	31018	31019	NC_035780.1	30535	31557
NC_035780.1	31024	31025	NC_035780.1	30535	31557
NC_035780.1	31025	31026	NC_035780.1	30535	31557


### 4h. Non-coding sequences

#### All 5x CpGs

In [54]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {nonCDS} \
| wc -l
!echo "all 5x CpG loci overlaps with nonCDS"

 2933517
all 5x CpG loci overlaps with nonCDS


In [55]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {nonCDS} \
> 2019-05-29-All5xCpGs-nonCDS.txt

In [56]:
!head 2019-05-29-All5xCpGs-nonCDS.txt

NC_007175.2	48	49	NC_007175.2	0	17244
NC_007175.2	49	50	NC_007175.2	0	17244
NC_007175.2	50	51	NC_007175.2	0	17244
NC_007175.2	51	52	NC_007175.2	0	17244
NC_007175.2	87	88	NC_007175.2	0	17244
NC_007175.2	88	89	NC_007175.2	0	17244
NC_007175.2	146	147	NC_007175.2	0	17244
NC_007175.2	147	148	NC_007175.2	0	17244
NC_007175.2	173	174	NC_007175.2	0	17244
NC_007175.2	192	193	NC_007175.2	0	17244


#### Methylated loci

In [57]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {nonCDS} \
| wc -l
!echo "methylated loci overlaps with nonCDS"

 2164988
methylated loci overlaps with nonCDS


In [58]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {nonCDS} \
> 2019-05-29-MethLoci-nonCDS.txt

In [59]:
!head 2019-05-29-MethLoci-nonCDS.txt

NC_035780.1	9253	9254	NC_035780.1	0	13577
NC_035780.1	9637	9638	NC_035780.1	0	13577
NC_035780.1	9657	9658	NC_035780.1	0	13577
NC_035780.1	10089	10090	NC_035780.1	0	13577
NC_035780.1	10331	10332	NC_035780.1	0	13577
NC_035780.1	11692	11693	NC_035780.1	0	13577
NC_035780.1	11706	11707	NC_035780.1	0	13577
NC_035780.1	11711	11712	NC_035780.1	0	13577
NC_035780.1	12686	12687	NC_035780.1	0	13577
NC_035780.1	12758	12759	NC_035780.1	0	13577


#### Sparsely methylated loci

In [60]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {nonCDS} \
| wc -l
!echo "sparsely methylated loci overlaps with nonCDS"

  375671
sparsely methylated loci overlaps with nonCDS


In [61]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {nonCDS} \
> 2019-05-29-SparseMethLoci-nonCDS.txt

In [62]:
!head 2019-05-29-SparseMethLoci-nonCDS.txt

NC_007175.2	1506	1507	NC_007175.2	0	17244
NC_007175.2	1820	1821	NC_007175.2	0	17244
NC_007175.2	2128	2129	NC_007175.2	0	17244
NC_007175.2	4841	4842	NC_007175.2	0	17244
NC_007175.2	13069	13070	NC_007175.2	0	17244
NC_035780.1	421	422	NC_035780.1	0	13577
NC_035780.1	1101	1102	NC_035780.1	0	13577
NC_035780.1	1540	1541	NC_035780.1	0	13577
NC_035780.1	3468	3469	NC_035780.1	0	13577
NC_035780.1	9254	9255	NC_035780.1	0	13577


#### Unmethylated loci

In [63]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {nonCDS} \
| wc -l
!echo "unmethylated loci overlaps with nonCDS"

  392858
unmethylated loci overlaps with nonCDS


In [64]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {nonCDS} \
> 2019-05-29-UnMethLoci-nonCDS.txt

In [65]:
!head 2019-05-29-UnMethLoci-nonCDS.txt

NC_007175.2	48	49	NC_007175.2	0	17244
NC_007175.2	49	50	NC_007175.2	0	17244
NC_007175.2	50	51	NC_007175.2	0	17244
NC_007175.2	51	52	NC_007175.2	0	17244
NC_007175.2	87	88	NC_007175.2	0	17244
NC_007175.2	88	89	NC_007175.2	0	17244
NC_007175.2	146	147	NC_007175.2	0	17244
NC_007175.2	147	148	NC_007175.2	0	17244
NC_007175.2	173	174	NC_007175.2	0	17244
NC_007175.2	192	193	NC_007175.2	0	17244


### 4i. Genes

#### All 5x CpGs

In [40]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {geneList} \
| wc -l
!echo "all 5x CpG loci overlaps with genes"

 3255049
all 5x CpG loci overlaps with genes


In [41]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {geneList} \
> 2019-05-29-All5xCpGs-Genes.txt

In [42]:
!head 2019-05-29-All5xCpGs-Genes.txt

NC_035780.1	28992	28993	NC_035780.1	28961	33324
NC_035780.1	29001	29002	NC_035780.1	28961	33324
NC_035780.1	29412	29413	NC_035780.1	28961	33324
NC_035780.1	30723	30724	NC_035780.1	28961	33324
NC_035780.1	30765	30766	NC_035780.1	28961	33324
NC_035780.1	30811	30812	NC_035780.1	28961	33324
NC_035780.1	30906	30907	NC_035780.1	28961	33324
NC_035780.1	30932	30933	NC_035780.1	28961	33324
NC_035780.1	30935	30936	NC_035780.1	28961	33324
NC_035780.1	31017	31018	NC_035780.1	28961	33324


In [44]:
!cut -f6 2019-05-29-All5xCpGs-Genes.txt| sort | uniq -c | wc -l
!echo "unique genes represented in overlaps"

   33126
unique genes represented in overlaps


#### Methylated loci

In [45]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {geneList} \
| wc -l
!echo "methylated loci overlaps with genes"

 2521653
methylated loci overlaps with genes


In [47]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {geneList} \
> 2019-05-29-MethLoci-Genes.txt

In [48]:
!head 2019-05-29-MethLoci-Genes.txt

NC_035780.1	29412	29413	NC_035780.1	28961	33324
NC_035780.1	87531	87532	NC_035780.1	85606	95254
NC_035780.1	87541	87542	NC_035780.1	85606	95254
NC_035780.1	87590	87591	NC_035780.1	85606	95254
NC_035780.1	87595	87596	NC_035780.1	85606	95254
NC_035780.1	100558	100559	NC_035780.1	99840	106460
NC_035780.1	100559	100560	NC_035780.1	99840	106460
NC_035780.1	100575	100576	NC_035780.1	99840	106460
NC_035780.1	100576	100577	NC_035780.1	99840	106460
NC_035780.1	100581	100582	NC_035780.1	99840	106460


In [52]:
!cut -f6 2019-05-29-MethLoci-Genes.txt| sort | uniq -c | wc -l
!echo "unique genes represented in overlaps"

   25496
unique genes represented in overlaps


#### Sparsely methylated loci

In [53]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {geneList} \
| wc -l
!echo "sparsely methylated loci overlaps with genes"

  317249
sparsely methylated loci overlaps with genes


In [54]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {geneList} \
> 2019-05-29-SparseMethLoci-Genes.txt

In [55]:
!head 2019-05-29-SparseMethLoci-Genes.txt

NC_035780.1	31078	31079	NC_035780.1	28961	33324
NC_035780.1	45142	45143	NC_035780.1	43111	66897
NC_035780.1	45542	45543	NC_035780.1	43111	66897
NC_035780.1	48914	48915	NC_035780.1	43111	66897
NC_035780.1	48928	48929	NC_035780.1	43111	66897
NC_035780.1	48940	48941	NC_035780.1	43111	66897
NC_035780.1	85755	85756	NC_035780.1	85606	95254
NC_035780.1	87599	87600	NC_035780.1	85606	95254
NC_035780.1	87607	87608	NC_035780.1	85606	95254
NC_035780.1	94754	94755	NC_035780.1	85606	95254


In [57]:
!cut -f6 2019-05-29-SparseMethLoci-Genes.txt| sort | uniq -c | wc -l
!echo "unique genes represesnted in overlaps"

   26953
unique genes represesnted in overlaps


#### Unmethylated loci

In [58]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {geneList} \
| wc -l
!echo "unmethylated loci overlaps with genes"

  416147
unmethylated loci overlaps with genes


In [59]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {geneList} \
> 2019-05-29-UnMethLoci-Genes.txt

In [60]:
!head 2019-05-29-UnMethLoci-Genes.txt

NC_035780.1	28992	28993	NC_035780.1	28961	33324
NC_035780.1	29001	29002	NC_035780.1	28961	33324
NC_035780.1	30723	30724	NC_035780.1	28961	33324
NC_035780.1	30765	30766	NC_035780.1	28961	33324
NC_035780.1	30811	30812	NC_035780.1	28961	33324
NC_035780.1	30906	30907	NC_035780.1	28961	33324
NC_035780.1	30932	30933	NC_035780.1	28961	33324
NC_035780.1	30935	30936	NC_035780.1	28961	33324
NC_035780.1	31017	31018	NC_035780.1	28961	33324
NC_035780.1	31018	31019	NC_035780.1	28961	33324


In [61]:
!cut -f6 2019-05-29-UnMethLoci-Genes.txt| sort | uniq -c | wc -l
!echo "unique genes represented in overlaps"

   27753
unique genes represented in overlaps


### 4j. Putative promoters

#### All 5x CpGs

In [18]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {putativePromoters} \
| wc -l
!echo "all 5x CpG loci overlaps with putative promoters"

  176156
all 5x CpG loci overlaps with putative promoters


In [19]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {putativePromoters} \
> 2019-05-29-All5xCpGs-Putative-Promoters.txt

In [20]:
!head 2019-05-29-All5xCpGs-Putative-Promoters.txt

NC_035780.1	27969	27970	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	27979	27980	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	28082	28083	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:1

#### Methylated loci

In [21]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {putativePromoters} \
| wc -l
!echo "methylated loci overlaps with putative promoters"

  106111
methylated loci overlaps with putative promoters


In [22]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {putativePromoters} \
> 2019-05-29-MethLoci-Putative-Promoters.txt

In [23]:
!head 2019-05-29-MethLoci-Putative-Promoters.txt

NC_035780.1	27969	27970	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	27979	27980	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	28082	28083	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:1

#### Sparsely methylated loci

In [24]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {putativePromoters} \
| wc -l
!echo "sparsely methylated loci overlaps with putative promoters"

   22870
sparsely methylated loci overlaps with putative promoters


In [25]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {putativePromoters} \
> 2019-05-29-SparseMethLoci-Putative-Promoters.txt

In [26]:
!head 2019-05-29-SparseMethLoci-Putative-Promoters.txt

NC_035780.1	95674	95675	NC_035780.1	Gnomon	mRNA	95255	96254	.	-	.	ID=rna4;Parent=gene3;Dbxref=GeneID:111112434,Genbank:XM_022449924.1;Name=XM_022449924.1;gbkey=mRNA;gene=LOC111112434;model_evidence=Supporting evidence includes similarity to: 7 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 13 samples with support for all annotated introns;product=homeobox protein Hox-B7-like;transcript_id=XM_022449924.1
NC_035780.1	99251	99252	NC_035780.1	Gnomon	mRNA	98840	99839	.	+	.	ID=rna5;Parent=gene4;Dbxref=GeneID:111120752,Genbank:XM_022461698.1;Name=XM_022461698.1;gbkey=mRNA;gene=LOC111120752;model_evidence=Supporting evidence includes similarity to: 10 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 27 samples with support for all annotated introns;product=ribulose-phosphate 3-epimerase-like;transcript_id=XM_022461698.1
NC_035780.1	232223	232224	NC_035780.1	Gnomon	mRNA	231965	232964	.	-	.	ID=r

#### Unmethylated loci

In [27]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {putativePromoters} \
| wc -l
!echo "unmethylated loci overlaps with putative promoters"

   47175
unmethylated loci overlaps with putative promoters


In [28]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {putativePromoters} \
> 2019-05-29-UnMethLoci-Putative-Promoters.txt

In [29]:
!head 2019-05-29-UnMethLoci-Putative-Promoters.txt

NC_035780.1	28859	28860	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	28924	28925	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	46515	46516	NC_035780.1	Gnomon	mRNA	46507	47506	.	-	.	ID=rna3;Parent=gene2;Dbxref=GeneID:1

### 4k. Transposable elements (all)

#### All 5x CpGs

In [62]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {transposableElementsAll} \
| wc -l
!echo "all 5x CpG loci overlaps with transposable elements (all)"

 1011883
all 5x CpG loci overlaps with transposable elements (all)


In [63]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {transposableElementsAll} \
> 2019-05-29-All5xCpGs-TE-All.txt

In [64]:
!head 2019-05-29-All5xCpGs-TE-All.txt

NC_007175.2	263	264	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	264	265	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	265	266	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	266	267	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	295	296	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	331	332	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	332	333	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	366	367	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	367	368	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	397	398

#### Methylated loci

In [65]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {transposableElementsAll} \
| wc -l
!echo "methylated loci overlaps with transposable elements (all)"

  755222
methylated loci overlaps with transposable elements (all)


In [66]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {transposableElementsAll} \
> 2019-05-29-MethLoci-TE-All.txt

In [67]:
!head 2019-05-29-MethLoci-TE-All.txt

NC_035780.1	9253	9254	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	19631	19632	NC_035780.1	RepeatMasker	similarity	19431	19866	23.3	-	.	Target "Motif:Crypton-N19_CGi" 580 1033
NC_035780.1	19741	19742	NC_035780.1	RepeatMasker	similarity	19431	19866	23.3	-	.	Target "Motif:Crypton-N19_CGi" 580 1033
NC_035780.1	37557	37558	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37581	37582	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37604	37605	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37611	37612	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37618	37619	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37622	37623	NC_035780.1	RepeatMasker	

#### Sparsely methylated loci

In [68]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {transposableElementsAll} \
| wc -l
!echo "sparsely methylated loci overlaps with transposable elements (all)"

  155293
sparsely methylated loci overlaps with transposable elements (all)


In [69]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {transposableElementsAll} \
> 2019-05-29-SparseMethLoci-TE-All.txt

In [70]:
!head 2019-05-29-SparseMethLoci-TE-All.txt

NC_007175.2	1820	1821	NC_007175.2	RepeatMasker	similarity	1728	1947	26.1	-	.	Target "Motif:REP-6_LMi" 14320 14534
NC_007175.2	2128	2129	NC_007175.2	RepeatMasker	similarity	2129	2367	20.5	-	.	Target "Motif:REP-6_LMi" 13886 14118
NC_035780.1	9254	9255	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9266	9267	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9267	9268	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9297	9298	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9298	9299	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9301	9302	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9302	9303	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780

#### Unmethylated loci

In [71]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {transposableElementsAll} \
| wc -l
!echo "unmethylated loci overlaps with transposable elements (all)"

  101368
unmethylated loci overlaps with transposable elements (all)


In [72]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {transposableElementsAll} \
> 2019-05-29-UnMethLoci-TE-All.txt

In [73]:
!head 2019-05-29-UnMethLoci-TE-All.txt

NC_007175.2	263	264	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	264	265	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	265	266	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	266	267	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	295	296	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	331	332	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	332	333	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	366	367	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	367	368	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	397	398

### 4l. Transposable elements (*C. gigas* only)

#### All 5x CpGs

In [74]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {transposableElementsCg} \
| wc -l
!echo "all 5x CpG loci overlaps with transposable elements (Cg)"

  767604
all 5x CpG loci overlaps with transposable elements (Cg)


In [75]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {transposableElementsCg} \
> 2019-05-29-All5xCpGs-TE-Cg.txt

In [76]:
!head 2019-05-29-All5xCpGs-TE-Cg.txt

NC_007175.2	1873	1874	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	1874	1875	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	1918	1919	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	1919	1920	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	2003	2004	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	2004	2005	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_035780.1	6036	6037	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	6109	6110	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	9253	9254	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.

#### Methylated loci

In [77]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {transposableElementsCg} \
| wc -l
!echo "methylated loci overlaps with transposable elements (Cg)"

  610208
methylated loci overlaps with transposable elements (Cg)


In [78]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {transposableElementsCg} \
> 2019-05-29-MethLoci-TE-Cg.txt

In [79]:
!head 2019-05-29-MethLoci-TE-Cg.txt

NC_035780.1	9253	9254	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	19631	19632	NC_035780.1	RepeatMasker	similarity	19431	19866	23.3	-	.	Target "Motif:Crypton-N19_CGi" 580 1033
NC_035780.1	19741	19742	NC_035780.1	RepeatMasker	similarity	19431	19866	23.3	-	.	Target "Motif:Crypton-N19_CGi" 580 1033
NC_035780.1	41723	41724	NC_035780.1	RepeatMasker	similarity	41713	41751	10.3	+	.	Target "Motif:Helitron-N10B_CGi" 258 296
NC_035780.1	41723	41724	NC_035780.1	RepeatMasker	similarity	41719	41776	 6.9	+	.	Target "Motif:Helitron-10_CGi" 282 358
NC_035780.1	73023	73024	NC_035780.1	RepeatMasker	similarity	72892	73822	28.6	-	.	Target "Motif:Kolobok-N4_CGi" 1 925
NC_035780.1	87531	87532	NC_035780.1	RepeatMasker	similarity	87526	87837	24.3	-	.	Target "Motif:DNA3-12_CGi" 60 378
NC_035780.1	87541	87542	NC_035780.1	RepeatMasker	similarity	87526	87837	24.3	-	.	Target "Motif:DNA3-12_CGi" 60 378
NC_035780.1	87590	87591	NC_035780.1	RepeatMasker	similarity	

#### Sparsely methylated loci

In [80]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {transposableElementsCg} \
| wc -l
!echo "sparsely methylated loci overlaps with transposable elements (Cg)"

  108858
sparsely methylated loci overlaps with transposable elements (Cg)


In [81]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {transposableElementsCg} \
> 2019-05-29-SparseMethLoci-TE-Cg.txt

In [82]:
!head 2019-05-29-SparseMethLoci-TE-Cg.txt

NC_035780.1	9254	9255	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9266	9267	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9267	9268	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9297	9298	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9298	9299	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9301	9302	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9302	9303	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	41739	41740	NC_035780.1	RepeatMasker	similarity	41713	41751	10.3	+	.	Target "Motif:Helitron-N10B_CGi" 258 296
NC_035780.1	41739	41740	NC_035780.1	RepeatMasker	similarity	41719	41776	 6.9	+	.	Target "Motif:Helitron-10_CGi" 282

#### Unmethylated loci

In [83]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {transposableElementsCg} \
| wc -l
!echo "unmethylated loci overlaps with transposable elements (Cg)"

   48538
unmethylated loci overlaps with transposable elements (Cg)


In [84]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {transposableElementsCg} \
> 2019-05-29-UnMethLoci-TE-Cg.txt

In [85]:
!head 2019-05-29-UnMethLoci-TE-Cg.txt

NC_007175.2	1873	1874	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	1874	1875	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	1918	1919	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	1919	1920	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	2003	2004	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	2004	2005	NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_035780.1	6036	6037	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	6109	6110	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	25242	25243	NC_035780.1	RepeatMasker	similarity	24971	26871	22.1

### 4m. lncRNA

#### All 5x CpGs

In [66]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {lncRNA} \
| wc -l
!echo "all 5x CpG loci overlaps with lncRNA"

   82671
all 5x CpG loci overlaps with lncRNA


In [67]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {lncRNA} \
> 2019-05-29-All5xCpGs-lncRNA.txt

In [68]:
!head 2019-05-29-All5xCpGs-lncRNA.txt

NC_035780.1	901982	901983	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57;Dbxref=GeneID:111111519,Genbank:XR_002636046.1;Name=XR_002636046.1;gbkey=ncRNA;gene=LOC111111519;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 20 samples with support for all annotated introns;product=uncharacterized LOC111111519;transcript_id=XR_002636046.1
NC_035780.1	902077	902078	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57;Dbxref=GeneID:111111519,Genbank:XR_002636046.1;Name=XR_002636046.1;gbkey=ncRNA;gene=LOC111111519;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 20 samples with support for all annotated introns;product=uncharacterized LOC111111519;transcript_id=XR_002636046.1
NC_035780.1	902078	902079	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57

#### Methylated loci

In [69]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {lncRNA} \
| wc -l
!echo "methylated loci overlaps with lncRNA"

   63588
methylated loci overlaps with lncRNA


In [70]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {lncRNA} \
> 2019-05-29-MethLoci-lncRNA.txt

In [71]:
!head 2019-05-29-MethLoci-lncRNA.txt

NC_035780.1	902092	902093	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57;Dbxref=GeneID:111111519,Genbank:XR_002636046.1;Name=XR_002636046.1;gbkey=ncRNA;gene=LOC111111519;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 20 samples with support for all annotated introns;product=uncharacterized LOC111111519;transcript_id=XR_002636046.1
NC_035780.1	902108	902109	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57;Dbxref=GeneID:111111519,Genbank:XR_002636046.1;Name=XR_002636046.1;gbkey=ncRNA;gene=LOC111111519;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 20 samples with support for all annotated introns;product=uncharacterized LOC111111519;transcript_id=XR_002636046.1
NC_035780.1	902109	902110	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57

#### Sparsely methylated loci

In [72]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {lncRNA} \
| wc -l
!echo "sparsely methylated loci overlaps with lncRNA"

    9337
sparsely methylated loci overlaps with lncRNA


In [73]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {lncRNA} \
> 2019-05-29-SparseMethLoci-lncRNA.txt

In [74]:
!head 2019-05-29-SparseMethLoci-lncRNA.txt

NC_035780.1	902077	902078	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57;Dbxref=GeneID:111111519,Genbank:XR_002636046.1;Name=XR_002636046.1;gbkey=ncRNA;gene=LOC111111519;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 20 samples with support for all annotated introns;product=uncharacterized LOC111111519;transcript_id=XR_002636046.1
NC_035780.1	902078	902079	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57;Dbxref=GeneID:111111519,Genbank:XR_002636046.1;Name=XR_002636046.1;gbkey=ncRNA;gene=LOC111111519;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 20 samples with support for all annotated introns;product=uncharacterized LOC111111519;transcript_id=XR_002636046.1
NC_035780.1	902091	902092	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57

#### Unmethylated loci

In [75]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {lncRNA} \
| wc -l
!echo "unmethylated loci overlaps with lncRNA"

    9746
unmethylated loci overlaps with lncRNA


In [76]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {lncRNA} \
> 2019-05-29-UnMethLoci-lncRNA.txt

In [77]:
!head 2019-05-29-UnMethLoci-lncRNA.txt

NC_035780.1	901982	901983	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57;Dbxref=GeneID:111111519,Genbank:XR_002636046.1;Name=XR_002636046.1;gbkey=ncRNA;gene=LOC111111519;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 20 samples with support for all annotated introns;product=uncharacterized LOC111111519;transcript_id=XR_002636046.1
NC_035780.1	902112	902113	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57;Dbxref=GeneID:111111519,Genbank:XR_002636046.1;Name=XR_002636046.1;gbkey=ncRNA;gene=LOC111111519;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 20 samples with support for all annotated introns;product=uncharacterized LOC111111519;transcript_id=XR_002636046.1
NC_035780.1	902578	902579	NC_035780.1	Gnomon	lnc_RNA	900326	903430	.	+	.	ID=rna105;Parent=gene57

### 4n. Intergenic regions

The intergenic regions are similar to the "no overlaps," but these would theoretically include transposable elements.

#### All 5x CpGs

In [78]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {intergenic} \
| wc -l
!echo "all 5x CpG loci overlaps with intergenic regions"

 1049088
all 5x CpG loci overlaps with intergenic regions


In [79]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {intergenic} \
> 2019-05-29-All5xCpGs-intergenic.txt

In [80]:
!head 2019-05-29-All5xCpGs-intergenic.txt

NC_007175.2	48	49	NC_007175.2	0	17244
NC_007175.2	49	50	NC_007175.2	0	17244
NC_007175.2	50	51	NC_007175.2	0	17244
NC_007175.2	51	52	NC_007175.2	0	17244
NC_007175.2	87	88	NC_007175.2	0	17244
NC_007175.2	88	89	NC_007175.2	0	17244
NC_007175.2	146	147	NC_007175.2	0	17244
NC_007175.2	147	148	NC_007175.2	0	17244
NC_007175.2	173	174	NC_007175.2	0	17244
NC_007175.2	192	193	NC_007175.2	0	17244


#### Methylated loci

In [81]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {intergenic} \
| wc -l
!echo "methylated loci overlaps with intergenic regions"

  660197
methylated loci overlaps with intergenic regions


In [82]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {intergenic} \
> 2019-05-29-MethLoci-intergenic.txt

In [83]:
!head 2019-05-29-MethLoci-intergenic.txt

NC_035780.1	9253	9254	NC_035780.1	0	13577
NC_035780.1	9637	9638	NC_035780.1	0	13577
NC_035780.1	9657	9658	NC_035780.1	0	13577
NC_035780.1	10089	10090	NC_035780.1	0	13577
NC_035780.1	10331	10332	NC_035780.1	0	13577
NC_035780.1	11692	11693	NC_035780.1	0	13577
NC_035780.1	11706	11707	NC_035780.1	0	13577
NC_035780.1	11711	11712	NC_035780.1	0	13577
NC_035780.1	12686	12687	NC_035780.1	0	13577
NC_035780.1	12758	12759	NC_035780.1	0	13577


#### Sparsely methylated loci

In [84]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {intergenic} \
| wc -l
!echo "sparsely methylated loci overlaps with intergenic regions"

  164528
sparsely methylated loci overlaps with intergenic regions


In [85]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {intergenic} \
> 2019-05-29-SparseMethLoci-intergenic.txt

In [86]:
!head 2019-05-29-SparseMethLoci-intergenic.txt

NC_007175.2	1506	1507	NC_007175.2	0	17244
NC_007175.2	1820	1821	NC_007175.2	0	17244
NC_007175.2	2128	2129	NC_007175.2	0	17244
NC_007175.2	4841	4842	NC_007175.2	0	17244
NC_007175.2	13069	13070	NC_007175.2	0	17244
NC_035780.1	421	422	NC_035780.1	0	13577
NC_035780.1	1101	1102	NC_035780.1	0	13577
NC_035780.1	1540	1541	NC_035780.1	0	13577
NC_035780.1	3468	3469	NC_035780.1	0	13577
NC_035780.1	9254	9255	NC_035780.1	0	13577


#### Unmethylated loci

In [87]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {intergenic} \
| wc -l
!echo "unmethylated loci overlaps with intergenic regions"

  224363
unmethylated loci overlaps with intergenic regions


In [88]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {intergenic} \
> 2019-05-29-UnMethLoci-intergenic.txt

In [89]:
!head 2019-05-29-UnMethLoci-intergenic.txt

NC_007175.2	48	49	NC_007175.2	0	17244
NC_007175.2	49	50	NC_007175.2	0	17244
NC_007175.2	50	51	NC_007175.2	0	17244
NC_007175.2	51	52	NC_007175.2	0	17244
NC_007175.2	87	88	NC_007175.2	0	17244
NC_007175.2	88	89	NC_007175.2	0	17244
NC_007175.2	146	147	NC_007175.2	0	17244
NC_007175.2	147	148	NC_007175.2	0	17244
NC_007175.2	173	174	NC_007175.2	0	17244
NC_007175.2	192	193	NC_007175.2	0	17244


### 4o. No overlaps

#### All 5x CpGs

In [30]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {all5xCpGs} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
| wc -l
!echo "all 5x CpG loci do not overlap with exons, introns, transposable elements (all), or putative promoters"

  603597
all 5x CpG loci do not overlap with exons, introns, transposable elements (all), or putative promoters


In [31]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {all5xCpGs} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
> 2019-05-29-All5xCpGs-NoOverlaps.txt

In [32]:
!head 2019-05-29-All5xCpGs-NoOverlaps.txt

NC_007175.2	48	49
NC_007175.2	49	50
NC_007175.2	50	51
NC_007175.2	51	52
NC_007175.2	87	88
NC_007175.2	88	89
NC_007175.2	146	147
NC_007175.2	147	148
NC_007175.2	173	174
NC_007175.2	192	193


#### Methylated loci

In [33]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {methylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
| wc -l
!echo "methylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters"

  372047
methylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters


In [34]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {methylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
> 2019-05-29-MethLoci-NoOverlaps.txt

In [35]:
!head 2019-05-29-MethLoci-NoOverlaps.txt

NC_035780.1	9637	9638
NC_035780.1	9657	9658
NC_035780.1	10089	10090
NC_035780.1	10331	10332
NC_035780.1	11692	11693
NC_035780.1	11706	11707
NC_035780.1	11711	11712
NC_035780.1	12686	12687
NC_035780.1	12758	12759
NC_035780.1	13486	13487


#### Sparsely methylated loci

In [36]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {sparselyMethylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
| wc -l
!echo "sparsely methylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters"

   84582
sparsely methylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters


In [37]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {sparselyMethylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
> 2019-05-29-SparseMethLoci-NoOverlaps.txt

In [38]:
!head 2019-05-29-SparseMethLoci-NoOverlaps.txt

NC_007175.2	1506	1507
NC_007175.2	4841	4842
NC_007175.2	13069	13070
NC_035780.1	421	422
NC_035780.1	1101	1102
NC_035780.1	1540	1541
NC_035780.1	3468	3469
NC_035780.1	9789	9790
NC_035780.1	9832	9833
NC_035780.1	9854	9855


#### Unmethylated loci

In [39]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {unmethylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
| wc -l
!echo "unmethylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters"

  146968
unmethylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters


In [40]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {unmethylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
> 2019-05-29-UnMethLoci-NoOverlaps.txt

In [41]:
!head 2019-05-29-UnMethLoci-NoOverlaps.txt

NC_007175.2	48	49
NC_007175.2	49	50
NC_007175.2	50	51
NC_007175.2	51	52
NC_007175.2	87	88
NC_007175.2	88	89
NC_007175.2	146	147
NC_007175.2	147	148
NC_007175.2	173	174
NC_007175.2	192	193


## 5. Identify methylation islands

To identify methylation islands using the method from Jeong et al. (2018), I need to define:

- starting size of the methylation window: 200 bp, 300 bp
- minimum fraction of methylated CpGs required within the window to be accepted: 0.02, 0.05, 0.10, 0.15, 0.20, 0.25, 0.27, 0.30
- step size to extend the accepted window as long as the mCpG fraction is met: 50 bp
- mCpG file: input with mCpG chromosome and bp position

### 5a. Create mCpG input file

In [ ]:
#Modify mCpG file by removing the third column that is not needed for methylation island analysis
!awk '{print $1"\t"$2}' 2019-04-09-All-5x-CpG-Loci-Methylated.bed > 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed

In [5]:
#Confirm file only has chromosome and start bp for mCpG
!head 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed

NC_035780.1	9253
NC_035780.1	9637
NC_035780.1	9657
NC_035780.1	10089
NC_035780.1	10331
NC_035780.1	11692
NC_035780.1	11706
NC_035780.1	11711
NC_035780.1	12686
NC_035780.1	12758


### 5b. Change mCpG fraction with 200 bp windows

In [7]:
#Identify methylation islands using 0.02 mCpG fraction (same as original paper)
! ./methyl_island_sliding_window.pl 200 0.02 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-200_0.02_50.tab

In [9]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-200_0.02_50.tab
!wc -l 2020-02-06-Methylation-Islands-200_0.02_50.tab

NC_035780.1	19901	20081	5
NC_035780.1	21693	21915	6
NC_035780.1	23585	23723	13
NC_035780.1	27826	28082	7
NC_035780.1	36000	36358	11
NC_035780.1	37557	37672	8
NC_035780.1	68011	68137	5
NC_035780.1	87531	87595	4
NC_035780.1	99242	99377	7
NC_035780.1	100558	101923	30
  119705 2020-02-06-Methylation-Islands-200_0.02_50.tab


In [31]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-200_0.02_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-200_0.02_50.tab

24777
4


In [86]:
#Identify methylation islands using 0.03 mCpG fraction
! ./methyl_island_sliding_window.pl 200 0.03 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-200_0.03_50.tab

In [87]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-200_0.03_50.tab
!wc -l 2020-02-06-Methylation-Islands-200_0.03_50.tab

NC_035780.1	23585	23723	13
NC_035780.1	27826	27979	6
NC_035780.1	36000	36046	6
NC_035780.1	37557	37672	8
NC_035780.1	99242	99377	7
NC_035780.1	100558	100975	16
NC_035780.1	101305	101465	6
NC_035780.1	102650	103702	36
NC_035780.1	105574	105697	6
NC_035780.1	115832	116009	7
  129006 2020-02-06-Methylation-Islands-200_0.03_50.tab


In [88]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-200_0.03_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-200_0.03_50.tab

8305
6


In [89]:
#Identify methylation islands using 0.04 mCpG fraction
! ./methyl_island_sliding_window.pl 200 0.04 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-200_0.04_50.tab

In [90]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-200_0.04_50.tab
!wc -l 2020-02-06-Methylation-Islands-200_0.04_50.tab

NC_035780.1	23585	23723	13
NC_035780.1	37557	37672	8
NC_035780.1	100558	100665	14
NC_035780.1	102796	103193	16
NC_035780.1	103268	103487	11
NC_035780.1	132694	132725	9
NC_035780.1	211497	211544	10
NC_035780.1	239676	239697	12
NC_035780.1	246023	246198	13
NC_035780.1	246529	246682	8
  113806 2020-02-06-Methylation-Islands-200_0.04_50.tab


In [91]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-200_0.04_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-200_0.04_50.tab

1682
8


In [66]:
#Identify methylation islands using 0.05 mCpG fraction
! ./methyl_island_sliding_window.pl 200 0.05 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-200_0.05_50.tab

In [67]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-200_0.05_50.tab
!wc -l 2020-02-06-Methylation-Islands-200_0.05_50.tab

NC_035780.1	23585	23723	13
NC_035780.1	100558	100665	14
NC_035780.1	102796	102998	13
NC_035780.1	211497	211544	10
NC_035780.1	239676	239697	12
NC_035780.1	246023	246198	13
NC_035780.1	246682	247287	39
NC_035780.1	250197	251369	64
NC_035780.1	252208	252391	11
NC_035780.1	253694	254047	20
   93229 2020-02-06-Methylation-Islands-200_0.05_50.tab


In [68]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-200_0.05_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-200_0.05_50.tab

1452
10


In [69]:
#Identify methylation islands using 0.02 mCpG fraction
! ./methyl_island_sliding_window.pl 200 0.10 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-200_0.10_50.tab

In [70]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-200_0.10_50.tab
!wc -l 2020-02-06-Methylation-Islands-200_0.10_50.tab

NC_035780.1	250321	250476	20
NC_035780.1	258914	259469	63
NC_035780.1	261610	262324	79
NC_035780.1	265186	265504	40
NC_035780.1	268658	268851	22
NC_035780.1	269593	269790	25
NC_035780.1	269906	270447	57
NC_035780.1	274533	274851	35
NC_035780.1	292546	292729	20
NC_035780.1	301695	301891	21
   18719 2020-02-06-Methylation-Islands-200_0.10_50.tab


In [71]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-200_0.10_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-200_0.10_50.tab

1167
20


In [10]:
#Identify methylation islands using 0.15 mCpG fraction (same percentage as overall genome methylation)
! ./methyl_island_sliding_window.pl 200 0.15 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-200_0.15_50.tab

In [11]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-200_0.15_50.tab
!wc -l 2020-02-06-Methylation-Islands-200_0.15_50.tab

NC_035780.1	261674	261823	32
NC_035780.1	261934	262160	39
NC_035780.1	269981	270179	34
NC_035780.1	369907	370328	70
NC_035780.1	389228	389585	61
NC_035780.1	405594	405792	32
NC_035780.1	575210	575409	31
NC_035780.1	604854	605022	31
NC_035780.1	780544	780740	32
NC_035780.1	966550	966748	30
    2453 2020-02-06-Methylation-Islands-200_0.15_50.tab


In [32]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-200_0.15_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-200_0.15_50.tab

177
30


In [16]:
#Identify methylation islands using 0.20 mCpG fraction
! ./methyl_island_sliding_window.pl 200 0.20 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-200_0.20_50.tab

In [17]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-200_0.20_50.tab
!wc -l 2020-02-06-Methylation-Islands-200_0.20_50.tab

NC_035780.1	369982	370180	40
NC_035780.1	389386	389585	40
NC_035780.1	1224242	1224471	53
NC_035780.1	2936196	2936385	43
NC_035780.1	7389637	7389836	45
NC_035780.1	8518124	8518369	52
NC_035780.1	10303310	10303509	41
NC_035780.1	11031936	11032122	41
NC_035780.1	12722855	12723123	60
NC_035780.1	13656765	13656999	51
     320 2020-02-06-Methylation-Islands-200_0.20_50.tab


In [33]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-200_0.20_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-200_0.20_50.tab

94
40


In [12]:
#Identify methylation islands using 0.25 mCpG fraction
! ./methyl_island_sliding_window.pl 200 0.25 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-200_0.25_50.tab

In [13]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-200_0.25_50.tab
!wc -l 2020-02-06-Methylation-Islands-200_0.25_50.tab

NC_035780.1	12722917	12723102	54
NC_035780.1	40413496	40413687	51
NC_035780.1	40786902	40787086	53
NC_035780.1	40862487	40862728	63
NC_035780.1	40863963	40864158	52
NC_035780.1	42061062	42061261	51
NC_035781.1	20940805	20940990	53
NC_035781.1	20952447	20952643	51
NC_035781.1	32151064	32151256	50
NC_035782.1	33824637	33824833	53
      37 2020-02-06-Methylation-Islands-200_0.25_50.tab


In [34]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-200_0.25_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-200_0.25_50.tab

63
50


In [22]:
#Identify methylation islands using 0.27 mCpG fraction
! ./methyl_island_sliding_window.pl 200 0.27 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-200_0.27_50.tab

In [23]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-200_0.27_50.tab
!wc -l 2020-02-06-Methylation-Islands-200_0.27_50.tab

NC_035780.1	12722917	12723102	54
NC_035780.1	40862512	40862707	57
NC_035782.1	33824643	33824838	54
NC_035782.1	34850304	34850499	54
NC_035782.1	34856648	34856843	54
NC_035787.1	61012904	61013089	55
NC_035787.1	62811875	62812071	54
NC_035789.1	5349692	5349885	54
       8 2020-02-06-Methylation-Islands-200_0.27_50.tab


In [29]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-200_0.27_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-200_0.27_50.tab

57
54


In [20]:
#Identify methylation islands using 0.30 mCpG fraction
! ./methyl_island_sliding_window.pl 200 0.30 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-200_0.30_50.tab

In [21]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-200_0.30_50.tab
!wc -l 2020-02-06-Methylation-Islands-200_0.30_50.tab

       0 2020-02-06-Methylation-Islands-200_0.30_50.tab


Obviously as the mCpG fraction increases, the number of methylation islands identified decreases. The differnece between maximum and minimum mCpG in a methylation islands decreases as methylation fraction increases.

### 5c. Change mCpG fraction with 300 bp windows

In [35]:
#Identify methylation islands using 0.02 mCpG fraction (same as original paper)
! ./methyl_island_sliding_window.pl 300 0.02 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-300_0.02_50.tab

In [36]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-300_0.02_50.tab
!wc -l 2020-02-06-Methylation-Islands-300_0.02_50.tab

NC_035780.1	19960	20231	6
NC_035780.1	21693	21915	6
NC_035780.1	23585	23723	13
NC_035780.1	27826	28082	7
NC_035780.1	36000	36358	11
NC_035780.1	37557	37672	8
NC_035780.1	99242	99377	7
NC_035780.1	100558	101923	30
NC_035780.1	102593	103702	37
NC_035780.1	105574	105842	9
   91756 2020-02-06-Methylation-Islands-300_0.02_50.tab


In [37]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-300_0.02_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-300_0.02_50.tab

24777
6


In [92]:
#Identify methylation islands using 0.03 mCpG fraction
! ./methyl_island_sliding_window.pl 300 0.03 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-300_0.03_50.tab

In [93]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-300_0.03_50.tab
!wc -l 2020-02-06-Methylation-Islands-300_0.03_50.tab

NC_035780.1	23585	23723	13
NC_035780.1	100558	100975	16
NC_035780.1	101408	101655	9
NC_035780.1	102593	103487	31
NC_035780.1	105574	105842	9
NC_035780.1	132694	132725	9
NC_035780.1	211497	211544	10
NC_035780.1	239676	239697	12
NC_035780.1	245847	246198	14
NC_035780.1	246529	247287	46
   91833 2020-02-06-Methylation-Islands-300_0.03_50.tab


In [94]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-300_0.03_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-300_0.03_50.tab

8305
9


In [95]:
#Identify methylation islands using 0.02 mCpG fraction (same as original paper)
! ./methyl_island_sliding_window.pl 300 0.04 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-300_0.04_50.tab

In [96]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-300_0.04_50.tab
!wc -l 2020-02-06-Methylation-Islands-300_0.04_50.tab

NC_035780.1	23585	23723	13
NC_035780.1	100558	100665	14
NC_035780.1	102650	102998	15
NC_035780.1	103193	103487	13
NC_035780.1	239676	239697	12
NC_035780.1	246023	246198	13
NC_035780.1	246529	247287	46
NC_035780.1	250197	251979	77
NC_035780.1	253694	254243	22
NC_035780.1	254295	254590	12
   74497 2020-02-06-Methylation-Islands-300_0.04_50.tab


In [97]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-300_0.04_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-300_0.04_50.tab

1682
12


In [72]:
#Identify methylation islands using 0.05 mCpG fraction (same as original paper)
! ./methyl_island_sliding_window.pl 300 0.05 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-300_0.05_50.tab

In [73]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-300_0.05_50.tab
!wc -l 2020-02-06-Methylation-Islands-300_0.05_50.tab

NC_035780.1	246682	247287	39
NC_035780.1	250197	251369	64
NC_035780.1	253694	254047	20
NC_035780.1	254488	255098	33
NC_035780.1	255802	256069	16
NC_035780.1	256185	256771	30
NC_035780.1	257471	258606	58
NC_035780.1	258744	260478	97
NC_035780.1	261610	263401	99
NC_035780.1	264570	265885	79
   53510 2020-02-06-Methylation-Islands-300_0.05_50.tab


In [74]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-300_0.05_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-300_0.05_50.tab

1452
15


In [78]:
#Identify methylation islands using 0.10 mCpG fraction (same as original paper)
! ./methyl_island_sliding_window.pl 300 0.10 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-300_0.10_50.tab

In [79]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-300_0.10_50.tab
!wc -l 2020-02-06-Methylation-Islands-300_0.10_50.tab

NC_035780.1	246810	247096	31
NC_035780.1	258914	259469	63
NC_035780.1	261610	262324	79
NC_035780.1	265098	265387	34
NC_035780.1	269493	269790	31
NC_035780.1	269846	270278	47
NC_035780.1	274533	274851	35
NC_035780.1	302340	302571	31
NC_035780.1	369554	369861	35
NC_035780.1	369907	370644	80
    6629 2020-02-06-Methylation-Islands-300_0.10_50.tab


In [80]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-300_0.10_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-300_0.10_50.tab

1167
30


In [38]:
#Identify methylation islands using 0.15 mCpG fraction (same percentage as overall genome methylation)
! ./methyl_island_sliding_window.pl 300 0.15 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-300_0.15_50.tab

In [39]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-300_0.15_50.tab
!wc -l 2020-02-06-Methylation-Islands-300_0.15_50.tab

NC_035780.1	261705	262004	47
NC_035780.1	369810	370308	76
NC_035780.1	389192	389585	65
NC_035780.1	1224200	1224471	54
NC_035780.1	1956315	1956664	53
NC_035780.1	2571108	2571453	56
NC_035780.1	2936102	2936385	47
NC_035780.1	7389553	7389900	53
NC_035780.1	7392779	7393125	58
NC_035780.1	8518124	8518424	55
     546 2020-02-06-Methylation-Islands-300_0.15_50.tab


In [40]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-300_0.15_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-300_0.15_50.tab

177
45


In [41]:
#Identify methylation islands using 0.20 mCpG fraction
! ./methyl_island_sliding_window.pl 300 0.20 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-300_0.20_50.tab

In [42]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-300_0.20_50.tab
!wc -l 2020-02-06-Methylation-Islands-300_0.20_50.tab

NC_035780.1	12722811	12723102	60
NC_035780.1	19448484	19448783	62
NC_035780.1	40413475	40413739	63
NC_035780.1	40862433	40862780	73
NC_035780.1	40863963	40864262	62
NC_035780.1	59707789	59708137	72
NC_035782.1	34850219	34850499	60
NC_035782.1	34855116	34855414	61
NC_035782.1	45369499	45369791	63
NC_035783.1	20473699	20473982	60
      20 2020-02-06-Methylation-Islands-300_0.20_50.tab


In [43]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-300_0.20_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-300_0.20_50.tab

94
60


In [44]:
#Identify methylation islands using 0.25 mCpG fraction
! ./methyl_island_sliding_window.pl 300 0.25 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-300_0.25_50.tab

In [45]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-300_0.25_50.tab
!wc -l 2020-02-06-Methylation-Islands-300_0.25_50.tab

       0 2020-02-06-Methylation-Islands-300_0.25_50.tab


It's interesting how increasing the window size leads to less identified methylation islands.

### 5d. Change step size with 500 bp windows

In [4]:
#Identify methylation islands using 0.02 mCpG fraction (same as original paper) but 25 bp step size
! ./methyl_island_sliding_window.pl 500 0.02 25 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-500_0.02_25.tab

In [5]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-500_0.02_25.tab
!wc -l 2020-02-06-Methylation-Islands-500_0.02_25.tab

NC_035780.1	23585	23723	13
NC_035780.1	36000	36358	11
NC_035780.1	100558	101923	30
NC_035780.1	102593	103702	37
NC_035780.1	115832	116304	11
NC_035780.1	211199	211544	11
NC_035780.1	239676	240134	13
NC_035780.1	245717	248838	63
NC_035780.1	250197	351003	2024
NC_035780.1	352791	353232	10
   64795 2020-02-06-Methylation-Islands-500_0.02_25.tab


In [6]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-500_0.02_25.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-500_0.02_25.tab

24777
10


In [90]:
#Filter by MI length and include MI length in a new column
!awk '{if ($3-$2 >= 500) { print $1"\t"$2"\t"$3"\t"$4"\t"$3-$2}}' 2020-02-06-Methylation-Islands-500_0.02_25.tab \
> 2020-02-06-Methylation-Islands-500_0.02_25-filtered.tab
!head 2020-02-06-Methylation-Islands-500_0.02_25-filtered.tab
! wc -l 2020-02-06-Methylation-Islands-500_0.02_25-filtered.tab

NC_035780.1	100558	101923	30	1365
NC_035780.1	102593	103702	37	1109
NC_035780.1	245717	248838	63	3121
NC_035780.1	250197	351003	2024	100806
NC_035780.1	353355	356963	89	3608
NC_035780.1	369554	378352	185	8798
NC_035780.1	380654	423774	1105	43120
NC_035780.1	449440	450158	23	718
NC_035780.1	471401	472285	31	884
NC_035780.1	529221	530454	26	1233
   36060 2020-02-06-Methylation-Islands-500_0.02_25-filtered.tab


In [28]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-500_0.02_25-filtered.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-500_0.02_25-filtered.tab

24777
11


In [22]:
#Identify methylation islands using 0.02 mCpG fraction (same as original paper)
! ./methyl_island_sliding_window.pl 500 0.02 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-500_0.02_50.tab

In [23]:
#chr, star, end, number mCpG
#Number of methylation islands
!head 2020-02-06-Methylation-Islands-500_0.02_50.tab
!wc -l 2020-02-06-Methylation-Islands-500_0.02_50.tab

NC_035780.1	23585	23723	13
NC_035780.1	36000	36358	11
NC_035780.1	100558	101923	30
NC_035780.1	102593	103702	37
NC_035780.1	115832	116304	11
NC_035780.1	211199	211544	11
NC_035780.1	239676	240134	13
NC_035780.1	245717	248838	63
NC_035780.1	250197	351003	2024
NC_035780.1	352791	353232	10
   63483 2020-02-06-Methylation-Islands-500_0.02_50.tab


In [21]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-500_0.02_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-500_0.02_50.tab

24777
10


In [91]:
#Filter by MI length and print MI length in a new column
!awk '{if ($3-$2 >= 500) { print $1"\t"$2"\t"$3"\t"$4"\t"$3-$2}}' 2020-02-06-Methylation-Islands-500_0.02_50.tab \
> 2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab
!head 2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab
! wc -l 2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab

NC_035780.1	100558	101923	30	1365
NC_035780.1	102593	103702	37	1109
NC_035780.1	245717	248838	63	3121
NC_035780.1	250197	351003	2024	100806
NC_035780.1	353355	356963	89	3608
NC_035780.1	369554	378352	185	8798
NC_035780.1	380654	423774	1105	43120
NC_035780.1	449440	450158	23	718
NC_035780.1	471401	472285	31	884
NC_035780.1	529221	530454	26	1233
   37063 2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab


In [26]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab

24777
11


## 5e. Create BEDfiles for IGV

In [29]:
#Identify files that need bedgraphs
!find *.tab

2020-02-06-Methylation-Islands-200_0.02_50.tab
2020-02-06-Methylation-Islands-200_0.03_50.tab
2020-02-06-Methylation-Islands-200_0.04_50.tab
2020-02-06-Methylation-Islands-200_0.05_50.tab
2020-02-06-Methylation-Islands-200_0.10_50.tab
2020-02-06-Methylation-Islands-200_0.15_50.tab
2020-02-06-Methylation-Islands-200_0.20_50.tab
2020-02-06-Methylation-Islands-200_0.25_50.tab
2020-02-06-Methylation-Islands-200_0.27_50.tab
2020-02-06-Methylation-Islands-200_0.30_50.tab
2020-02-06-Methylation-Islands-300_0.02_50.tab
2020-02-06-Methylation-Islands-300_0.03_50.tab
2020-02-06-Methylation-Islands-300_0.04_50.tab
2020-02-06-Methylation-Islands-300_0.05_50.tab
2020-02-06-Methylation-Islands-300_0.10_50.tab
2020-02-06-Methylation-Islands-300_0.15_50.tab
2020-02-06-Methylation-Islands-300_0.20_50.tab
2020-02-06-Methylation-Islands-300_0.25_50.tab
2020-02-06-Methylation-Islands-500_0.02_25-filtered.tab
2020-02-06-Methylation-Islands-500_0.02_25.tab
2020-02-06-Methylation-Islands-500_0.02_50-filtered

In [92]:
%%bash
for f in *.tab
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
done

In [93]:
#Remove bedgraphs that correspond to files with no MI
!rm 2020-02-06-Methylation-Islands-200_0.30_50.tab.bed 2020-02-06-Methylation-Islands-300_0.25_50.tab.bed

In [94]:
#See what files remain
!find *tab.bed

2020-02-06-Methylation-Islands-200_0.02_50.tab.bed
2020-02-06-Methylation-Islands-200_0.03_50.tab.bed
2020-02-06-Methylation-Islands-200_0.04_50.tab.bed
2020-02-06-Methylation-Islands-200_0.05_50.tab.bed
2020-02-06-Methylation-Islands-200_0.10_50.tab.bed
2020-02-06-Methylation-Islands-200_0.15_50.tab.bed
2020-02-06-Methylation-Islands-200_0.20_50.tab.bed
2020-02-06-Methylation-Islands-200_0.25_50.tab.bed
2020-02-06-Methylation-Islands-200_0.27_50.tab.bed
2020-02-06-Methylation-Islands-300_0.02_50.tab.bed
2020-02-06-Methylation-Islands-300_0.03_50.tab.bed
2020-02-06-Methylation-Islands-300_0.04_50.tab.bed
2020-02-06-Methylation-Islands-300_0.05_50.tab.bed
2020-02-06-Methylation-Islands-300_0.10_50.tab.bed
2020-02-06-Methylation-Islands-300_0.15_50.tab.bed
2020-02-06-Methylation-Islands-300_0.20_50.tab.bed
2020-02-06-Methylation-Islands-500_0.02_25-filtered.tab.bed
2020-02-06-Methylation-Islands-500_0.02_25.tab.bed
2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab.bed
2020-02-06-Me

In [95]:
#Check the file to ensure loop worked
!head 2020-02-06-Methylation-Islands-200_0.02_50.tab.bed

NC_035780.1	19901	20081
NC_035780.1	21693	21915
NC_035780.1	23585	23723
NC_035780.1	27826	28082
NC_035780.1	36000	36358
NC_035780.1	37557	37672
NC_035780.1	68011	68137
NC_035780.1	87531	87595
NC_035780.1	99242	99377
NC_035780.1	100558	101923


### 5f. Characterize MI overlaps with genome feature tracks

In [96]:
methylationIslands = "2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab.bed"

In [97]:
!head {methylationIslands}
!wc -l {methylationIslands}

NC_035780.1	100558	101923
NC_035780.1	102593	103702
NC_035780.1	245717	248838
NC_035780.1	250197	351003
NC_035780.1	353355	356963
NC_035780.1	369554	378352
NC_035780.1	380654	423774
NC_035780.1	449440	450158
NC_035780.1	471401	472285
NC_035780.1	529221	530454
   37063 2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab.bed


#### Location of MI in genome feature files

##### Exons

In [98]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {exonList} \
| wc -l
!echo "MI overlaps with exons"

   22705
MI overlaps with exons


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {exonList} \
> 2020-02-06-MI-Exons.txt

In [100]:
!head 2020-02-06-MI-Exons.txt

NC_035780.1	100558	101923	NC_035780.1	100554	100661	103
NC_035780.1	245717	248838	NC_035780.1	246019	246220	201
NC_035780.1	245717	248838	NC_035780.1	247019	247125	106
NC_035780.1	245717	248838	NC_035780.1	245532	245878	161
NC_035780.1	250197	351003	NC_035780.1	250285	250608	323
NC_035780.1	250197	351003	NC_035780.1	252747	253042	295
NC_035780.1	250197	351003	NC_035780.1	258108	259494	1386
NC_035780.1	250197	351003	NC_035780.1	263244	265531	2287
NC_035780.1	250197	351003	NC_035780.1	263245	265531	2286
NC_035780.1	250197	351003	NC_035780.1	266196	266755	559


##### Introns

In [99]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {intronList} \
| wc -l
!echo "MI overlaps with introns"

   28730
MI overlaps with introns


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {intronList} \
> 2020-02-06-MI-Introns.txt

In [101]:
!head 2020-02-06-MI-Introns.txt

NC_035780.1	100558	101923	NC_035780.1	100661	104928	1262
NC_035780.1	102593	103702	NC_035780.1	100661	104928	1109
NC_035780.1	245717	248838	NC_035780.1	245878	246018	140
NC_035780.1	245717	248838	NC_035780.1	246220	247018	798
NC_035780.1	245717	248838	NC_035780.1	247125	250284	1713
NC_035780.1	250197	351003	NC_035780.1	247125	250284	87
NC_035780.1	250197	351003	NC_035780.1	250608	252746	2138
NC_035780.1	250197	351003	NC_035780.1	259494	261477	1983
NC_035780.1	250197	351003	NC_035780.1	262168	263243	1075
NC_035780.1	250197	351003	NC_035780.1	265531	266195	664


##### Exon UTR

In [112]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {exonUTR} \
| wc -l
!echo "MI overlaps with exon UTR"

    8649
MI overlaps with exon UTR


In [113]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {exonUTR} \
> 2020-02-06-MI-exonUTR.txt

In [114]:
!head 2020-02-06-MI-exonUTR.txt

NC_035780.1	245717	248838	NC_035780.1	Gnomon	exon	245532	245768	.	-	.	ID=id70;Parent=rna19;Dbxref=GeneID:111109452,Genbank:XM_022445568.1;gbkey=mRNA;gene=LOC111109452;product=sulfotransferase 1C4-like;transcript_id=XM_022445568.1	51
NC_035780.1	250197	351003	NC_035780.1	Gnomon	exon	252907	253042	.	-	.	ID=id66;Parent=rna19;Dbxref=GeneID:111109452,Genbank:XM_022445568.1;gbkey=mRNA;gene=LOC111109452;product=sulfotransferase 1C4-like;transcript_id=XM_022445568.1	136
NC_035780.1	250197	351003	NC_035780.1	Gnomon	exon	258108	258985	.	-	.	ID=id75;Parent=rna20;Dbxref=GeneID:111124802,Genbank:XM_022468012.1;gbkey=mRNA;gene=LOC111124802;product=uncharacterized LOC111124802%2C transcript variant X3;transcript_id=XM_022468012.1	878
NC_035780.1	250197	351003	NC_035780.1	Gnomon	exon	261478	261665	.	-	.	ID=id80;Parent=rna21;Dbxref=GeneID:111124802,Genbank:XM_022468021.1;gbkey=mRNA;gene=LOC111124802;product=uncharacterized LOC111124802%2C transcript variant X4;transcript_id=XM_022468021.1	188
NC_035780

##### mRNA

In [167]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {mRNAList} \
| wc -l
!echo "MI overlaps with mRNA"

   29805
MI overlaps with mRNA


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {mRNAList} \
> 2020-02-06-MI-mRNA.txt

In [104]:
!head 2020-02-06-MI-mRNA.txt

NC_035780.1	100558	101923	NC_035780.1	Gnomon	mRNA	99840	106460	.	+	.	ID=rna5;Parent=gene4;Dbxref=GeneID:111120752,Genbank:XM_022461698.1;Name=XM_022461698.1;gbkey=mRNA;gene=LOC111120752;model_evidence=Supporting evidence includes similarity to: 10 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 27 samples with support for all annotated introns;product=ribulose-phosphate 3-epimerase-like;transcript_id=XM_022461698.1	1365
NC_035780.1	102593	103702	NC_035780.1	Gnomon	mRNA	99840	106460	.	+	.	ID=rna5;Parent=gene4;Dbxref=GeneID:111120752,Genbank:XM_022461698.1;Name=XM_022461698.1;gbkey=mRNA;gene=LOC111120752;model_evidence=Supporting evidence includes similarity to: 10 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 27 samples with support for all annotated introns;product=ribulose-phosphate 3-epimerase-like;transcript_id=XM_022461698.1	1109
NC_035780.1	245717	248838	NC_035780.1	Gnomon	mRNA	

##### Coding sequences

In [115]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {CDS} \
| wc -l
!echo "MI overlaps with CDS"

   20872
MI overlaps with CDS


In [116]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {CDS} \
> 2020-02-06-MI-CDS.txt

In [117]:
!head 2020-02-06-MI-CDS.txt

NC_035780.1	100558	101923	NC_035780.1	100554	100661	103
NC_035780.1	245717	248838	NC_035780.1	245769	245878	109
NC_035780.1	245717	248838	NC_035780.1	246019	246220	201
NC_035780.1	245717	248838	NC_035780.1	247019	247125	106
NC_035780.1	250197	351003	NC_035780.1	250285	250608	323
NC_035780.1	250197	351003	NC_035780.1	252747	252906	159
NC_035780.1	250197	351003	NC_035780.1	258986	259494	508
NC_035780.1	250197	351003	NC_035780.1	264903	265531	628
NC_035780.1	250197	351003	NC_035780.1	264903	265531	628
NC_035780.1	250197	351003	NC_035780.1	266454	266755	301


##### Non-coding sequences

In [118]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {nonCDS} \
| wc -l
!echo "MI overlaps with nonCDS"

   35932
MI overlaps with nonCDS


In [119]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {nonCDS} \
> 2020-02-06-MI-nonCDS.txt

In [120]:
!head 2020-02-06-MI-nonCDS.txt

NC_035780.1	100558	101923	NC_035780.1	100661	104928	1262
NC_035780.1	102593	103702	NC_035780.1	100661	104928	1109
NC_035780.1	245717	248838	NC_035780.1	245878	246018	140
NC_035780.1	245717	248838	NC_035780.1	246220	247018	798
NC_035780.1	245717	248838	NC_035780.1	247125	250284	1713
NC_035780.1	250197	351003	NC_035780.1	247125	250284	87
NC_035780.1	250197	351003	NC_035780.1	250608	252746	2138
NC_035780.1	250197	351003	NC_035780.1	253042	258107	5065
NC_035780.1	250197	351003	NC_035780.1	259494	261477	1983
NC_035780.1	250197	351003	NC_035780.1	262168	263243	1075


##### Genes

In [102]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {geneList} \
| wc -l
!echo "MI overlaps with genes"

   30773
MI overlaps with genes


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {geneList} \
> 2020-02-06-MI-Genes.txt

In [103]:
!head 2020-02-06-MI-Genes.txt

NC_035780.1	100558	101923	NC_035780.1	99840	106460	1365
NC_035780.1	102593	103702	NC_035780.1	99840	106460	1109
NC_035780.1	245717	248838	NC_035780.1	245532	253042	3121
NC_035780.1	250197	351003	NC_035780.1	273173	278473	5300
NC_035780.1	250197	351003	NC_035780.1	281547	293861	12314
NC_035780.1	250197	351003	NC_035780.1	245532	253042	2845
NC_035780.1	250197	351003	NC_035780.1	297131	311654	14523
NC_035780.1	250197	351003	NC_035780.1	315522	340261	24739
NC_035780.1	250197	351003	NC_035780.1	341638	349379	7741
NC_035780.1	250197	351003	NC_035780.1	349425	360957	1578


##### Putative promoters

In [108]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {putativePromoters} \
| wc -l
!echo "MI overlaps with promoter"

    4217
MI overlaps with promoter


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {putativePromoters} \
> 2020-02-06-MI-Promoter.txt

In [109]:
!head 2020-02-06-MI-Promoter.txt

NC_035780.1	250197	351003	NC_035780.1	Gnomon	mRNA	253043	254042	.	-	.	ID=rna19;Parent=gene16;Dbxref=GeneID:111109452,Genbank:XM_022445568.1;Name=XM_022445568.1;gbkey=mRNA;gene=LOC111109452;model_evidence=Supporting evidence includes similarity to: 14 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 26 samples with support for all annotated introns;product=sulfotransferase 1C4-like;transcript_id=XM_022445568.1	1000
NC_035780.1	250197	351003	NC_035780.1	Gnomon	mRNA	272173	273172	.	+	.	ID=rna25;Parent=gene18;Dbxref=GeneID:111101273,Genbank:XM_022433714.1;Name=XM_022433714.1;gbkey=mRNA;gene=LOC111101273;model_evidence=Supporting evidence includes similarity to: 1 EST%2C 4 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 27 samples with support for all annotated introns;product=mitochondrial inner membrane protein COX18-like;transcript_id=XM_022433714.1	1000
NC_035780.1	250197	351003	NC_03578

##### Transposable Elements (All)

In [105]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {transposableElementsAll} \
| wc -l
!echo "MI overlaps with TE (all)"

   25085
MI overlaps with TE (all)


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {transposableElementsAll} \
> 2020-02-06-MI-TEall.txt

In [106]:
!head 2020-02-06-MI-TEall.txt

NC_035780.1	102593	103702	NC_035780.1	RepeatMasker	similarity	102567	102619	21.8	+	.	Target "Motif:(AACAA)n" 1 55	26
NC_035780.1	102593	103702	NC_035780.1	RepeatMasker	similarity	103110	103178	23.9	+	.	Target "Motif:A-rich" 1 69	69
NC_035780.1	102593	103702	NC_035780.1	RepeatMasker	similarity	103644	103888	14.3	+	.	Target "Motif:BivaMD-SINE1_CrVi" 87 337	59
NC_035780.1	245717	248838	NC_035780.1	RepeatMasker	similarity	246222	246289	17.8	+	.	Target "Motif:(TATAATA)n" 1 69	68
NC_035780.1	250197	351003	NC_035780.1	RepeatMasker	similarity	250877	250911	23.3	+	.	Target "Motif:(GTA)n" 1 35	35
NC_035780.1	250197	351003	NC_035780.1	RepeatMasker	similarity	253139	253198	 0.0	+	.	Target "Motif:(TC)n" 1 60	60
NC_035780.1	250197	351003	NC_035780.1	RepeatMasker	similarity	255032	255159	14.5	-	.	Target "Motif:BivaMD-SINE1_CrVi" 211 337	128
NC_035780.1	250197	351003	NC_035780.1	RepeatMasker	similarity	255167	255254	15.9	+	.	Target "Motif:BEL2_Cis_int-int" 193 269	88
NC_035780.1	250197	351003	NC_03578

##### lncRNA

In [121]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {lncRNA} \
| wc -l
!echo "MI overlaps with lncRNA"

     949
MI overlaps with lncRNA


In [122]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {lncRNA} \
> 2020-02-06-MI-lncRNA.txt

In [123]:
!head 2020-02-06-MI-lncRNA.txt

NC_035780.1	1437386	1438420	NC_035780.1	Gnomon	lnc_RNA	1432944	1458091	.	+	.	ID=rna135;Parent=gene76;Dbxref=GeneID:111135942,Genbank:XR_002639675.1;Name=XR_002639675.1;gbkey=ncRNA;gene=LOC111135942;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 4 samples with support for all annotated introns;product=uncharacterized LOC111135942;transcript_id=XR_002639675.1	1034
NC_035780.1	1442453	1443010	NC_035780.1	Gnomon	lnc_RNA	1432944	1458091	.	+	.	ID=rna135;Parent=gene76;Dbxref=GeneID:111135942,Genbank:XR_002639675.1;Name=XR_002639675.1;gbkey=ncRNA;gene=LOC111135942;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 4 samples with support for all annotated introns;product=uncharacterized LOC111135942;transcript_id=XR_002639675.1	557
NC_035780.1	1444645	1445887	NC_035780.1	Gnomon	lnc_RNA	1432944	1458091	.	+	.	ID=r

##### Intergenic regions

In [124]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {intergenic} \
| wc -l
!echo "MI overlaps with intergenic regions"

   10302
MI overlaps with intergenic regions


In [125]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {intergenic} \
> 2020-02-06-MI-intergenic.txt

In [126]:
!head 2020-02-06-MI-intergenic.txt

NC_035780.1	250197	351003	NC_035780.1	253042	258107	5065
NC_035780.1	250197	351003	NC_035780.1	272839	273172	333
NC_035780.1	250197	351003	NC_035780.1	311654	315521	3867
NC_035780.1	250197	351003	NC_035780.1	340261	341637	1376
NC_035780.1	250197	351003	NC_035780.1	349379	349424	45
NC_035780.1	250197	351003	NC_035780.1	278473	281546	3073
NC_035780.1	250197	351003	NC_035780.1	293861	297130	3269
NC_035780.1	369554	378352	NC_035780.1	370670	372173	1503
NC_035780.1	369554	378352	NC_035780.1	376974	380453	1378
NC_035780.1	380654	423774	NC_035780.1	409280	409476	196


##### No overlaps

In [110]:
!{bedtoolsDirectory}intersectBed \
-v \
-a {methylationIslands} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters}  \
| wc -l
!echo "MI do not overlap with exons, introns, transposable elements (all), or putative promoters"

    1154
MI do not overlap with exons, introns, transposable elements (all), or putative promoters


In [ ]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {methylationIslands} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
> 2020-02-06-MI-No-Overlap.txt

In [111]:
!head 2020-02-06-MI-No-Overlap.txt

NC_035780.1	677431	678698
NC_035780.1	1036866	1038149
NC_035780.1	1192587	1194166
NC_035780.1	1342185	1342815
NC_035780.1	1373756	1374322
NC_035780.1	1382865	1383827
NC_035780.1	1386325	1387383
NC_035780.1	1467860	1468374
NC_035780.1	1469922	1471147
NC_035780.1	1531502	1532753


#### Location of individual genome features in MI

##### Exons

In [127]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {exonList} \
-b {methylationIslands} \
| wc -l
!echo "exon overlaps with MI"

  240133
exon overlaps with MI


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {exonList} \
-b {methylationIslands} \
> 2020-02-06-Exons-MI.txt

In [128]:
!head 2020-02-06-Exons-MI.txt

NC_035780.1	100554	100661	NC_035780.1	100558	101923	103
NC_035780.1	245532	245878	NC_035780.1	245717	248838	161
NC_035780.1	246019	246220	NC_035780.1	245717	248838	201
NC_035780.1	247019	247125	NC_035780.1	245717	248838	106
NC_035780.1	250285	250608	NC_035780.1	250197	351003	323
NC_035780.1	252747	253042	NC_035780.1	250197	351003	295
NC_035780.1	258108	259494	NC_035780.1	250197	351003	1386
NC_035780.1	261478	262168	NC_035780.1	250197	351003	690
NC_035780.1	263244	265531	NC_035780.1	250197	351003	2287
NC_035780.1	263245	265531	NC_035780.1	250197	351003	2286


##### Introns

In [129]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {intronList} \
-b {methylationIslands} \
| wc -l
!echo "intron overlaps with MI"

   92472
intron overlaps with MI


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {intronList} \
-b {methylationIslands} \
> 2020-02-06-Introns-MI.txt

In [130]:
!head 2020-02-06-Introns-MI.txt

NC_035780.1	100661	104928	NC_035780.1	100558	101923	1262
NC_035780.1	100661	104928	NC_035780.1	102593	103702	1109
NC_035780.1	245878	246018	NC_035780.1	245717	248838	140
NC_035780.1	246220	247018	NC_035780.1	245717	248838	798
NC_035780.1	247125	250284	NC_035780.1	245717	248838	1713
NC_035780.1	247125	250284	NC_035780.1	250197	351003	87
NC_035780.1	250608	252746	NC_035780.1	250197	351003	2138
NC_035780.1	259494	261477	NC_035780.1	250197	351003	1983
NC_035780.1	262168	263243	NC_035780.1	250197	351003	1075
NC_035780.1	265531	266195	NC_035780.1	250197	351003	664


##### Exon UTR

In [139]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {exonUTR} \
-b {methylationIslands} \
| wc -l
!echo "exonUTR overlaps with MI"

   30827
exonUTR overlaps with MI


In [140]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {exonUTR} \
-b {methylationIslands} \
> 2020-02-06-exonUTR-MI.txt

In [141]:
!head 2020-02-06-exonUTR-MI.txt

NC_035780.1	Gnomon	exon	245532	245768	.	-	.	ID=id70;Parent=rna19;Dbxref=GeneID:111109452,Genbank:XM_022445568.1;gbkey=mRNA;gene=LOC111109452;product=sulfotransferase 1C4-like;transcript_id=XM_022445568.1	NC_035780.1	245717	248838	51
NC_035780.1	Gnomon	exon	252907	253042	.	-	.	ID=id66;Parent=rna19;Dbxref=GeneID:111109452,Genbank:XM_022445568.1;gbkey=mRNA;gene=LOC111109452;product=sulfotransferase 1C4-like;transcript_id=XM_022445568.1	NC_035780.1	250197	351003	136
NC_035780.1	Gnomon	exon	258108	258985	.	-	.	ID=id75;Parent=rna20;Dbxref=GeneID:111124802,Genbank:XM_022468012.1;gbkey=mRNA;gene=LOC111124802;product=uncharacterized LOC111124802%2C transcript variant X3;transcript_id=XM_022468012.1	NC_035780.1	250197	351003	878
NC_035780.1	Gnomon	exon	261478	261665	.	-	.	ID=id80;Parent=rna21;Dbxref=GeneID:111124802,Genbank:XM_022468021.1;gbkey=mRNA;gene=LOC111124802;product=uncharacterized LOC111124802%2C transcript variant X4;transcript_id=XM_022468021.1	NC_035780.1	250197	351003	188
NC_035780

##### mRNA

In [168]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {mRNAList} \
-b {methylationIslands} \
| wc -l
!echo "mRNA overlaps with MI"

   29483
mRNA overlaps with MI


In [169]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {mRNAList} \
-b {methylationIslands} \
> 2020-02-06-mRNA-MI.txt

In [170]:
!head 2020-02-06-mRNA-MI.txt

NC_035780.1	Gnomon	mRNA	99840	106460	.	+	.	ID=rna5;Parent=gene4;Dbxref=GeneID:111120752,Genbank:XM_022461698.1;Name=XM_022461698.1;gbkey=mRNA;gene=LOC111120752;model_evidence=Supporting evidence includes similarity to: 10 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 27 samples with support for all annotated introns;product=ribulose-phosphate 3-epimerase-like;transcript_id=XM_022461698.1	NC_035780.1	100558	101923	1365
NC_035780.1	Gnomon	mRNA	99840	106460	.	+	.	ID=rna5;Parent=gene4;Dbxref=GeneID:111120752,Genbank:XM_022461698.1;Name=XM_022461698.1;gbkey=mRNA;gene=LOC111120752;model_evidence=Supporting evidence includes similarity to: 10 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 27 samples with support for all annotated introns;product=ribulose-phosphate 3-epimerase-like;transcript_id=XM_022461698.1	NC_035780.1	102593	103702	1109
NC_035780.1	Gnomon	mRNA	245532	253042	.	-	.	ID=rna

##### Coding sequences

In [142]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {CDS} \
-b {methylationIslands} \
| wc -l
!echo "CDS overlaps with MI"

  226237
CDS overlaps with MI


In [143]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {CDS} \
-b {methylationIslands} \
> 2020-02-06-CDS-MI.txt

In [144]:
!head 2020-02-06-CDS-MI.txt

NC_035780.1	100554	100661	NC_035780.1	100558	101923	103
NC_035780.1	245769	245878	NC_035780.1	245717	248838	109
NC_035780.1	246019	246220	NC_035780.1	245717	248838	201
NC_035780.1	247019	247125	NC_035780.1	245717	248838	106
NC_035780.1	250285	250608	NC_035780.1	250197	351003	323
NC_035780.1	252747	252906	NC_035780.1	250197	351003	159
NC_035780.1	258986	259494	NC_035780.1	250197	351003	508
NC_035780.1	261666	262168	NC_035780.1	250197	351003	502
NC_035780.1	264903	265531	NC_035780.1	250197	351003	628
NC_035780.1	264903	265531	NC_035780.1	250197	351003	628


##### Non-coding sequences

In [145]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {nonCDS} \
-b {methylationIslands} \
| wc -l
!echo "nonCDS overlaps with MI"

   98103
nonCDS overlaps with MI


In [146]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {nonCDS} \
-b {methylationIslands} \
> 2020-02-06-nonCDS-MI.txt

In [147]:
!head 2020-02-06-nonCDS-MI.txt

NC_035780.1	100661	104928	NC_035780.1	100558	101923	1262
NC_035780.1	100661	104928	NC_035780.1	102593	103702	1109
NC_035780.1	245878	246018	NC_035780.1	245717	248838	140
NC_035780.1	246220	247018	NC_035780.1	245717	248838	798
NC_035780.1	247125	250284	NC_035780.1	245717	248838	1713
NC_035780.1	247125	250284	NC_035780.1	250197	351003	87
NC_035780.1	250608	252746	NC_035780.1	250197	351003	2138
NC_035780.1	253042	258107	NC_035780.1	250197	351003	5065
NC_035780.1	259494	261477	NC_035780.1	250197	351003	1983
NC_035780.1	262168	263243	NC_035780.1	250197	351003	1075


##### Genes

In [131]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {geneList} \
-b {methylationIslands} \
| wc -l
!echo "gene overlaps with MI"

   15009
gene overlaps with MI


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {geneList} \
-b {methylationIslands} \
> 2020-02-06-Genes-MI.txt

In [132]:
!head 2020-02-06-Genes-MI.txt

NC_035780.1	Gnomon	gene	99840	106460	.	+	.	ID=gene4;Dbxref=GeneID:111120752;Name=LOC111120752;gbkey=Gene;gene=LOC111120752;gene_biotype=protein_coding	NC_035780.1	100558	101923	1365
NC_035780.1	Gnomon	gene	99840	106460	.	+	.	ID=gene4;Dbxref=GeneID:111120752;Name=LOC111120752;gbkey=Gene;gene=LOC111120752;gene_biotype=protein_coding	NC_035780.1	102593	103702	1109
NC_035780.1	Gnomon	gene	245532	253042	.	-	.	ID=gene16;Dbxref=GeneID:111109452;Name=LOC111109452;gbkey=Gene;gene=LOC111109452;gene_biotype=protein_coding	NC_035780.1	245717	248838	3121
NC_035780.1	Gnomon	gene	245532	253042	.	-	.	ID=gene16;Dbxref=GeneID:111109452;Name=LOC111109452;gbkey=Gene;gene=LOC111109452;gene_biotype=protein_coding	NC_035780.1	250197	351003	2845
NC_035780.1	Gnomon	gene	258108	272839	.	-	.	ID=gene17;Dbxref=GeneID:111124802;Name=LOC111124802;gbkey=Gene;gene=LOC111124802;gene_biotype=protein_coding	NC_035780.1	250197	351003	14732
NC_035780.1	Gnomon	gene	273173	278473	.	+	.	ID=gene18;Dbxref=GeneID:111101273;Name=

##### Putative promoters

In [137]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {putativePromoters} \
-b {methylationIslands} \
| wc -l
!echo "promoter overlaps with MI"

    8846
promoter overlaps with MI


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {promoterTrack} \
-b {methylationIslands} \
> 2020-02-06-Promoter-MI.txt

In [138]:
!head 2020-02-06-Promoter-MI.txt

NC_035780.1	Gnomon	mRNA	272173	273172	.	+	.	ID=rna25;Parent=gene18;Dbxref=GeneID:111101273,Genbank:XM_022433714.1;Name=XM_022433714.1;gbkey=mRNA;gene=LOC111101273;model_evidence=Supporting evidence includes similarity to: 1 EST%2C 4 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 27 samples with support for all annotated introns;product=mitochondrial inner membrane protein COX18-like;transcript_id=XM_022433714.1	NC_035780.1	250197	351003	1000
NC_035780.1	Gnomon	mRNA	280547	281546	.	+	.	ID=rna26;Parent=gene19;Dbxref=GeneID:111101250,Genbank:XM_022433686.1;Name=XM_022433686.1;gbkey=mRNA;gene=LOC111101250;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 13 samples with support for all annotated introns;product=uncharacterized LOC111101250%2C transcript variant X1;transcript_id=XM_022433686.1	NC_035780.1	250197	351003	1000
NC_035780.1	Gnomon	mRNA	2

##### Transposable Elements (All)

In [135]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {transposableElementsAll} \
-b {methylationIslands} \
| wc -l
!echo "TE overlaps with MI"

  107926
TE overlaps with MI


In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {transposableElementsAll} \
-b {methylationIslands} \
> 2020-02-06-TEall-MI.txt

In [136]:
!head 2020-02-06-TEall-MI.txt

NC_035780.1	RepeatMasker	similarity	102567	102619	21.8	+	.	Target "Motif:(AACAA)n" 1 55	NC_035780.1	102593	103702	26
NC_035780.1	RepeatMasker	similarity	103110	103178	23.9	+	.	Target "Motif:A-rich" 1 69	NC_035780.1	102593	103702	69
NC_035780.1	RepeatMasker	similarity	103644	103888	14.3	+	.	Target "Motif:BivaMD-SINE1_CrVi" 87 337	NC_035780.1	102593	103702	59
NC_035780.1	RepeatMasker	similarity	246222	246289	17.8	+	.	Target "Motif:(TATAATA)n" 1 69	NC_035780.1	245717	248838	68
NC_035780.1	RepeatMasker	similarity	250877	250911	23.3	+	.	Target "Motif:(GTA)n" 1 35	NC_035780.1	250197	351003	35
NC_035780.1	RepeatMasker	similarity	253139	253198	 0.0	+	.	Target "Motif:(TC)n" 1 60	NC_035780.1	250197	351003	60
NC_035780.1	RepeatMasker	similarity	255032	255159	14.5	-	.	Target "Motif:BivaMD-SINE1_CrVi" 211 337	NC_035780.1	250197	351003	128
NC_035780.1	RepeatMasker	similarity	255167	255254	15.9	+	.	Target "Motif:BEL2_Cis_int-int" 193 269	NC_035780.1	250197	351003	88
NC_035780.1	RepeatMasker	similarit

##### lncRNA

In [148]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {lncRNA} \
-b {methylationIslands} \
| wc -l
!echo "lncRNA overlaps with MI"

    1108
lncRNA overlaps with MI


In [149]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {lncRNA} \
-b {methylationIslands} \
> 2020-02-06-lncRNA-MI.txt

In [150]:
!head 2020-02-06-lncRNA-MI.txt

NC_035780.1	Gnomon	lnc_RNA	1432944	1458091	.	+	.	ID=rna135;Parent=gene76;Dbxref=GeneID:111135942,Genbank:XR_002639675.1;Name=XR_002639675.1;gbkey=ncRNA;gene=LOC111135942;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 4 samples with support for all annotated introns;product=uncharacterized LOC111135942;transcript_id=XR_002639675.1	NC_035780.1	1437386	1438420	1034
NC_035780.1	Gnomon	lnc_RNA	1432944	1458091	.	+	.	ID=rna135;Parent=gene76;Dbxref=GeneID:111135942,Genbank:XR_002639675.1;Name=XR_002639675.1;gbkey=ncRNA;gene=LOC111135942;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 4 samples with support for all annotated introns;product=uncharacterized LOC111135942;transcript_id=XR_002639675.1	NC_035780.1	1442453	1443010	557
NC_035780.1	Gnomon	lnc_RNA	1432944	1458091	.	+	.	ID=rna135;Parent=gene76;Dbxref=G

##### Intergenic regions

In [151]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {intergenic} \
-b {methylationIslands} \
| wc -l
!echo "CDS overlaps with intergenic regions"

    8526
CDS overlaps with intergenic regions


In [152]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {intergenic} \
-b {methylationIslands} \
> 2020-02-06-intergenic-MI.txt

In [153]:
!head 2020-02-06-intergenic-MI.txt

NC_035780.1	253042	258107	NC_035780.1	250197	351003	5065
NC_035780.1	272839	273172	NC_035780.1	250197	351003	333
NC_035780.1	278473	281546	NC_035780.1	250197	351003	3073
NC_035780.1	293861	297130	NC_035780.1	250197	351003	3269
NC_035780.1	311654	315521	NC_035780.1	250197	351003	3867
NC_035780.1	340261	341637	NC_035780.1	250197	351003	1376
NC_035780.1	349379	349424	NC_035780.1	250197	351003	45
NC_035780.1	370670	372173	NC_035780.1	369554	378352	1503
NC_035780.1	376974	380453	NC_035780.1	369554	378352	1378
NC_035780.1	392035	394982	NC_035780.1	380654	423774	2947
